In [1]:
#!/usr/bin/env python3
"""
NSE Options Analyzer - Combined PCR & Max Pain Calculator

Features:
1. Scrapes NSE option chain data for equities and indices
2. Calculates PCR (Put-Call Ratio) for all symbols
3. Calculates Max Pain for each option chain
4. Generates comprehensive summary reports
5. Robust error handling for symbols without options data

Output: Desktop/nse_options_analysis/
"""

import os
import sys
import time
import random
import traceback
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Dict, Optional

import requests
import pandas as pd
import matplotlib.pyplot as plt

# ==========================
# CONFIGURATION
# ==========================
# Output directory (new folder name)
BASE_DIR = os.path.join(os.path.expanduser("~"), "Desktop", "nse_options_analysis")

# NSE URLs
WARMUP_URL = "https://www.nseindia.com/option-chain"
EQUITY_OC_URL = "https://www.nseindia.com/api/option-chain-equities?symbol={symbol}"
INDEX_OC_URL = "https://www.nseindia.com/api/option-chain-indices?symbol={symbol}"
INDEX_OC_URL_ALT = "https://www.nseindia.com/api/option-chain-indices/?symbol={symbol}"

# Full equities ticker list
RAW_TICKERS = [
    'NSE:360ONE', 'NSE:ABB', 'NSE:ADANIENSOL', 'NSE:ADANIENT', 'NSE:ADANIGREEN', 'NSE:ADANIPORTS',
    'NSE:ATGL', 'NSE:ABCAPITAL', 'NSE:ABFRL', 'NSE:ALKEM', 'NSE:AMBER', 'NSE:AMBUJACEM',
    'NSE:ANGELONE', 'NSE:APLAPOLLO', 'NSE:APOLLOHOSP', 'NSE:ASHOKLEY', 'NSE:ASIANPAINT',
    'NSE:ASTRAL', 'NSE:AUBANK', 'NSE:AUROPHARMA', 'NSE:DMART', 'NSE:AXISBANK', 'NSE:BAJAJ-AUTO',
    'NSE:BAJFINANCE', 'NSE:BAJAJFINSV', 'NSE:BANDHANBNK', 'NSE:BANKBARODA', 'NSE:BANKINDIA',
    'NSE:BDL', 'NSE:BEL', 'NSE:BHARATFORG', 'NSE:BHEL', 'NSE:BPCL', 'NSE:BHARTIARTL',
    'NSE:BIOCON', 'NSE:BLUESTARCO', 'NSE:BOSCHLTD', 'NSE:BRITANNIA', 'NSE:BSE', 'NSE:CANBK',
    'NSE:CDSL', 'NSE:CESC', 'NSE:CGPOWER', 'NSE:CHOLAFIN', 'NSE:CIPLA', 'NSE:COALINDIA',
    'NSE:COFORGE', 'NSE:COLPAL', 'NSE:CAMS', 'NSE:CONCOR', 'NSE:CROMPTON', 'NSE:CUMMINSIND',
    'NSE:CYIENT', 'NSE:DABUR', 'NSE:DALBHARAT', 'NSE:DELHIVERY', 'NSE:DIVISLAB', 'NSE:DIXON',
    'NSE:DLF', 'NSE:DRREDDY', 'NSE:EICHERMOT', 'NSE:ETERNAL', 'NSE:EXIDEIND', 'NSE:FORTIS',
    'NSE:NYKAA', 'NSE:GAIL', 'NSE:GLENMARK', 'NSE:GMRAIRPORT', 'NSE:GODREJCP', 'NSE:GODREJPROP',
    'NSE:GRANULES', 'NSE:GRASIM', 'NSE:HAVELLS', 'NSE:HCLTECH', 'NSE:HDFCAMC', 'NSE:HDFCBANK',
    'NSE:HDFCLIFE', 'NSE:HEROMOTOCO', 'NSE:HFCL', 'NSE:HINDALCO', 'NSE:HAL', 'NSE:HINDPETRO',
    'NSE:HINDUNILVR', 'NSE:HINDZINC', 'NSE:HUDCO', 'NSE:ICICIBANK', 'NSE:ICICIGI', 'NSE:ICICIPRULI',
    'NSE:IDEA', 'NSE:IDFCFIRSTB', 'NSE:IIFL', 'NSE:INDIANB', 'NSE:IEX', 'NSE:IOC', 'NSE:IRCTC',
    'NSE:IRFC', 'NSE:IREDA', 'NSE:IGL', 'NSE:INDUSTOWER', 'NSE:INDUSINDBK', 'NSE:NAUKRI',
    'NSE:INFY', 'NSE:INOXWIND', 'NSE:INDIGO', 'NSE:IRB', 'NSE:ITC', 'NSE:JSL', 'NSE:JINDALSTEL',
    'NSE:JIOFIN', 'NSE:JSWENERGY', 'NSE:JSWSTEEL', 'NSE:JUBLFOOD', 'NSE:KALYANKJIL', 'NSE:KAYNES',
    'NSE:KEI', 'NSE:KFINTECH', 'NSE:KOTAKBANK', 'NSE:KPITTECH', 'NSE:LTF', 'NSE:LT', 'NSE:LAURUSLABS',
    'NSE:LICHSGFIN', 'NSE:LICI', 'NSE:LTIM', 'NSE:LUPIN', 'NSE:LODHA', 'NSE:MANAPPURAM',
    'NSE:MANKIND', 'NSE:MARICO', 'NSE:MARUTI', 'NSE:MFSL', 'NSE:MAXHEALTH', 'NSE:MAZDOCK',
    'NSE:MOTHERSON', 'NSE:MPHASIS', 'NSE:MCX', 'NSE:MUTHOOTFIN', 'NSE:NATIONALUM', 'NSE:NBCC',
    'NSE:NCC', 'NSE:NESTLEIND', 'NSE:NHPC', 'NSE:NMDC', 'NSE:NTPC', 'NSE:NUVAMA',
    'NSE:OBEROIRLTY', 'NSE:ONGC', 'NSE:OIL', 'NSE:PAYTM', 'NSE:OFSS', 'NSE:PAGEIND', 'NSE:PATANJALI',
    'NSE:POLICYBZR', 'NSE:PERSISTENT', 'NSE:PETRONET', 'NSE:PGEL', 'NSE:PIIND', 'NSE:PIDILITIND',
    'NSE:PPLPHARMA', 'NSE:PNBHOUSING', 'NSE:POLYCAB', 'NSE:POONAWALLA', 'NSE:PFC', 'NSE:POWERGRID',
    'NSE:PRESTIGE', 'NSE:PNB', 'NSE:RVNL', 'NSE:RBLBANK', 'NSE:RELIANCE', 'NSE:RECLTD', 'NSE:SBICARD',
    'NSE:SBILIFE', 'NSE:SHREECEM', 'NSE:SHRIRAMFIN', 'NSE:SIEMENS', 'NSE:SJVN', 'NSE:SOLARINDS',
    'NSE:SONACOMS', 'NSE:SRF', 'NSE:SBIN', 'NSE:SAIL', 'NSE:SUNPHARMA', 'NSE:SUPREMEIND', 'NSE:SUZLON',
    'NSE:SYNGENE', 'NSE:TATACHEM', 'NSE:TCS', 'NSE:TATACONSUM', 'NSE:TATAELXSI', 'NSE:TATAMOTORS',
    'NSE:TATAPOWER', 'NSE:TATASTEEL', 'NSE:TATATECH', 'NSE:TECHM', 'NSE:FEDERALBNK', 'NSE:INDHOTEL',
    'NSE:PHOENIXLTD', 'NSE:TITAGARH', 'NSE:TITAN', 'NSE:TORNTPHARM', 'NSE:TORNTPOWER', 'NSE:TRENT',
    'NSE:TIINDIA', 'NSE:TVSMOTOR', 'NSE:ULTRACEMCO', 'NSE:UNIONBANK', 'NSE:UNITDSPR', 'NSE:UNOMINDA',
    'NSE:UPL', 'NSE:VBL', 'NSE:VEDL', 'NSE:VOLTAS', 'NSE:WIPRO', 'NSE:YESBANK', 'NSE:ZYDUSLIFE'
]

# Indices to process
INDEX_LIST = ["NIFTY", "BANKNIFTY"]
INDEX_SYMBOLS = set(INDEX_LIST)

# Analysis settings
ENABLE_PLOTS = False  # Set to True to generate Max Pain plots
VERBOSE_ERRORS = False  # Set to True to see detailed error traces

# ==========================
# HTTP HELPERS
# ==========================
def new_session():
    """Create a new session with proper NSE headers"""
    s = requests.Session()
    s.headers.update({
        "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/115.0.0.0 Safari/537.36"),
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Referer": "https://www.nseindia.com/option-chain",
        "Host": "www.nseindia.com",
        "Cache-Control": "no-cache",
        "Pragma": "no-cache",
    })
    return s

def warm_cookies(session, retries=5, sleep_min=1.0, sleep_max=2.0):
    """
    Warm up session cookies by visiting NSE homepage.
    This is CRITICAL - NSE requires valid cookies before API access.
    """
    for i in range(retries):
        try:
            # Visit main page first
            resp = session.get(WARMUP_URL, timeout=15, allow_redirects=True)
            
            if resp.status_code == 200:
                # Also visit the derivatives page for better cookie setup
                time.sleep(random.uniform(0.3, 0.8))
                session.get("https://www.nseindia.com/get-quotes/derivatives", timeout=15)
                return True
            elif resp.status_code in [301, 302, 307, 308]:
                # Follow redirects manually if needed
                if 'Location' in resp.headers:
                    session.get(resp.headers['Location'], timeout=15)
                    return True
        except requests.exceptions.Timeout:
            print(f"  ⏱ Warmup attempt {i+1}/{retries}: Timeout, retrying...")
        except requests.exceptions.ConnectionError:
            print(f"  🌐 Warmup attempt {i+1}/{retries}: Connection error, retrying...")
        except Exception as e:
            if VERBOSE_ERRORS:
                print(f"  ⚠ Warmup attempt {i+1}/{retries}: {type(e).__name__}")
        
        # Exponential backoff
        wait_time = random.uniform(sleep_min, sleep_max) * (i + 1)
        time.sleep(wait_time)
    
    return False

def fetch_option_chain(session, symbol, is_index=False, retries=3, backoff=1.5):
    """
    Fetch NSE option-chain JSON for the given symbol.
    Returns the parsed JSON data or None if symbol has no options data.
    """
    urls = []
    if is_index:
        urls = [INDEX_OC_URL.format(symbol=symbol), INDEX_OC_URL_ALT.format(symbol=symbol)]
    else:
        urls = [EQUITY_OC_URL.format(symbol=symbol)]

    for url in urls:
        for attempt in range(retries):
            try:
                resp = session.get(url, timeout=15)
                
                if resp.status_code == 200:
                    try:
                        data = resp.json()
                        # Check if response has valid option data
                        if not data.get("records") or not data["records"].get("data"):
                            # No options data available for this symbol
                            return None
                        return data
                    except Exception:
                        # Non-JSON response, likely blocked
                        warm_cookies(session)
                        time.sleep(backoff * (attempt + 1))
                        continue
                        
                elif resp.status_code == 404:
                    # Symbol doesn't have options data
                    return None
                    
                elif resp.status_code in (429, 503, 403):
                    # Rate-limited/blocked - backoff and retry
                    time.sleep(backoff * (attempt + 1))
                    continue
                    
                else:
                    # Other HTTP errors
                    time.sleep(backoff * (attempt + 1))
                    
            except requests.exceptions.Timeout:
                time.sleep(backoff * (attempt + 1))
            except Exception as e:
                if VERBOSE_ERRORS:
                    print(f"  Fetch attempt {attempt+1} failed for {symbol}: {e}")
                time.sleep(backoff * (attempt + 1))
    
    # All retries exhausted - symbol likely has no options or network issue
    return None

# ==========================
# DATA PROCESSING HELPERS
# ==========================
def ensure_dir(path):
    """Create directory if it doesn't exist"""
    os.makedirs(path, exist_ok=True)

def strip_prefix(t):
    """Remove 'NSE:' prefix from ticker"""
    return t.split(":", 1)[1] if ":" in t else t

def parse_rows_for_expiry(data, expiry):
    """Extract option chain rows for a specific expiry date"""
    rows = []
    for rec in data.get("records", {}).get("data", []):
        if rec.get("expiryDate") == expiry:
            strike = rec.get("strikePrice")
            ce = rec.get("CE", {}) or {}
            pe = rec.get("PE", {}) or {}
            rows.append({
                "Strike": strike,
                "Expiry": expiry,
                "Call_OI": ce.get("openInterest", 0),
                "Call_LTP": ce.get("lastPrice", None),
                "Put_OI": pe.get("openInterest", 0),
                "Put_LTP": pe.get("lastPrice", None),
                "CE_BidQty": ce.get("bidQty"),
                "CE_BidPrice": ce.get("bidprice"),
                "CE_AskQty": ce.get("askQty"),
                "CE_AskPrice": ce.get("askPrice"),
                "PE_BidQty": pe.get("bidQty"),
                "PE_BidPrice": pe.get("bidprice"),
                "PE_AskQty": pe.get("askQty"),
                "PE_AskPrice": pe.get("askPrice"),
                "CE_IV": ce.get("impliedVolatility"),
                "PE_IV": pe.get("impliedVolatility"),
                "Underlying": ce.get("underlying") or pe.get("underlying"),
                "UnderlyingValue": ce.get("underlyingValue") or pe.get("underlyingValue"),
            })
    return rows

def compute_pcr(rows):
    """
    Calculate Put-Call Ratio (PCR)
    PCR = Total Put OI / Total Call OI
    """
    total_call_oi = sum((r.get("Call_OI") or 0) for r in rows)
    total_put_oi = sum((r.get("Put_OI") or 0) for r in rows)
    pcr = (total_put_oi / total_call_oi) if total_call_oi else None
    return total_call_oi, total_put_oi, (round(pcr, 6) if pcr is not None else None)

# ==========================
# MAX PAIN CALCULATION
# ==========================
def calculate_max_pain(df: pd.DataFrame) -> Optional[Dict]:
    """
    Calculate Max Pain strike price for an option chain.
    
    Max Pain Theory: The strike price at which option writers (sellers) 
    experience minimum total loss at expiration.
    
    Returns dict with: max_pain_strike, min_loss, or None if calculation fails
    """
    try:
        # Ensure required columns exist
        required = ['Strike', 'Call_OI', 'Put_OI']
        if not all(col in df.columns for col in required):
            return None
        
        # Clean data
        df = df.copy()
        df = df.dropna(subset=required)
        if df.empty:
            return None
        
        strikes = df['Strike'].values
        call_oi = df['Call_OI'].values
        put_oi = df['Put_OI'].values
        
        # Skip if all OI is zero
        if call_oi.sum() == 0 and put_oi.sum() == 0:
            return None
        
        total_losses = []
        # Calculate total loss at each potential expiry price
        for expiry_strike in strikes:
            # Call writers lose when price > strike: (price - strike) × call_oi
            call_loss = ((expiry_strike - strikes).clip(min=0) * call_oi).sum()
            # Put writers lose when price < strike: (strike - price) × put_oi
            put_loss = ((strikes - expiry_strike).clip(min=0) * put_oi).sum()
            total_losses.append(call_loss + put_loss)
        
        # Find strike with minimum loss
        min_loss_idx = total_losses.index(min(total_losses))
        max_pain_strike = int(strikes[min_loss_idx])
        min_loss = total_losses[min_loss_idx]
        
        return {
            'max_pain_strike': max_pain_strike,
            'min_loss': min_loss
        }
    except Exception as e:
        if VERBOSE_ERRORS:
            print(f"  ⚠ Max Pain calculation error: {e}")
        return None

def plot_max_pain(df: pd.DataFrame, title: str, save_path: str):
    """Generate and save Max Pain visualization"""
    try:
        strikes = df['Strike'].values
        call_oi = df['Call_OI'].values
        put_oi = df['Put_OI'].values
        
        total_losses = []
        for expiry_strike in strikes:
            call_loss = ((expiry_strike - strikes).clip(min=0) * call_oi).sum()
            put_loss = ((strikes - expiry_strike).clip(min=0) * put_oi).sum()
            total_losses.append(call_loss + put_loss)
        
        plt.figure(figsize=(10, 6))
        plt.bar(strikes, total_losses, edgecolor='black', alpha=0.7)
        plt.xlabel("Strike Price")
        plt.ylabel("Total Loss (to Option Writers)")
        plt.title(title)
        plt.xticks(rotation=45)
        
        max_pain_idx = total_losses.index(min(total_losses))
        max_pain = int(strikes[max_pain_idx])
        plt.axvline(max_pain, color='red', linestyle='--', linewidth=2, 
                    label=f"Max Pain = {max_pain}")
        plt.legend()
        plt.grid(alpha=0.3)
        
        plt.savefig(save_path, bbox_inches="tight", dpi=150)
        plt.close()
    except Exception as e:
        if VERBOSE_ERRORS:
            print(f"  ⚠ Plot generation error: {e}")

# ==========================
# EQUITY PROCESSING
# ==========================
def process_symbol(session, raw_symbol, target_expiries, out_base):
    """
    Process a single equity symbol:
    1. Fetch option chain data
    2. Save per-expiry CSVs
    3. Calculate PCR and Max Pain for each expiry
    
    Returns list of results or empty list if symbol has no options
    """
    symbol = strip_prefix(raw_symbol).upper()
    is_index = symbol in INDEX_SYMBOLS

    # First attempt
    data = fetch_option_chain(session, symbol, is_index=is_index)
    
    # If failed, try once more with cookie refresh
    if data is None:
        warm_cookies(session)
        time.sleep(random.uniform(0.5, 1.0))
        data = fetch_option_chain(session, symbol, is_index=is_index)
    
    # If still no data, symbol doesn't have options
    if data is None:
        return []  # Return empty list instead of raising error

    found_expiries = set(data.get("records", {}).get("expiryDates", []))
    if not found_expiries:
        return []
    
    out_dir = os.path.join(out_base, symbol)
    ensure_dir(out_dir)

    pcr_maxpain_out = []
    
    for expiry in target_expiries:
        if expiry not in found_expiries:
            continue
        
        rows = parse_rows_for_expiry(data, expiry)
        if not rows:
            continue
        
        df = pd.DataFrame(rows).sort_values("Strike").reset_index(drop=True)
        
        # Save option chain CSV
        fname = f"{symbol}_{expiry}.csv".replace("/", "-")
        out_path = os.path.join(out_dir, fname)
        try:
            df.to_csv(out_path, index=False)
        except Exception as e:
            if VERBOSE_ERRORS:
                print(f"[WARN] Error writing {out_path}: {e}")
        
        # Calculate PCR
        tcoi, tpoi, pcr = compute_pcr(rows)
        
        # Calculate Max Pain
        max_pain_result = calculate_max_pain(df)
        max_pain_strike = max_pain_result['max_pain_strike'] if max_pain_result else None
        
        # Generate Max Pain plot if enabled
        if ENABLE_PLOTS and max_pain_result:
            plot_path = os.path.join(out_dir, f"{symbol}_{expiry}_MaxPain.png".replace("/", "-"))
            plot_title = f"{symbol} - {expiry} Max Pain Analysis"
            plot_max_pain(df, plot_title, plot_path)
        
        pcr_maxpain_out.append({
            "Symbol": symbol,
            "Expiry": expiry,
            "Total_Call_OI": tcoi,
            "Total_Put_OI": tpoi,
            "PCR": pcr,
            "Max_Pain_Strike": max_pain_strike
        })
    
    return pcr_maxpain_out

def process_symbol_threadsafe(raw_symbol, target_expiries, out_base):
    """Thread-safe wrapper for processing symbols"""
    s = new_session()
    warm_cookies(s)
    time.sleep(random.uniform(0.2, 0.7))
    
    try:
        return process_symbol(s, raw_symbol, target_expiries, out_base)
    except Exception as e:
        symbol = strip_prefix(raw_symbol)
        print(f"  ⚠ {symbol}: No options data available or fetch error")
        if VERBOSE_ERRORS:
            traceback.print_exc()
        return []

# ==========================
# INDEX PROCESSING
# ==========================
def process_index_all_expiries(session, symbol, out_base):
    """
    Process all expiries for an index:
    1. Fetch option chain data
    2. Save per-expiry CSVs
    3. Calculate PCR and Max Pain for each expiry
    4. Generate master summary file
    """
    symbol = symbol.upper()
    data = fetch_option_chain(session, symbol, is_index=True)
    
    if data is None:
        raise RuntimeError(f"No option data available for index {symbol}")

    expiries = data.get("records", {}).get("expiryDates", [])
    if not expiries:
        raise RuntimeError(f"No expiries found for index {symbol}")

    out_dir = os.path.join(out_base, symbol)
    ensure_dir(out_dir)

    pcr_maxpain_rows = []
    
    for expiry in expiries:
        rows = parse_rows_for_expiry(data, expiry)
        
        if not rows:
            # Create empty CSV for transparency
            df_empty = pd.DataFrame(columns=[
                "Strike", "Expiry", "Call_OI", "Call_LTP", "Put_OI", "Put_LTP",
                "CE_BidQty", "CE_BidPrice", "CE_AskQty", "CE_AskPrice",
                "PE_BidQty", "PE_BidPrice", "PE_AskQty", "PE_AskPrice",
                "CE_IV", "PE_IV", "Underlying", "UnderlyingValue"
            ])
            out_path = os.path.join(out_dir, f"{symbol}_{expiry}.csv".replace("/", "-"))
            try:
                df_empty.to_csv(out_path, index=False)
            except Exception as e:
                if VERBOSE_ERRORS:
                    print(f"[WARN] Error writing empty CSV {out_path}: {e}")
            
            pcr_maxpain_rows.append({
                "Expiry": expiry,
                "Total_Call_OI": 0,
                "Total_Put_OI": 0,
                "PCR": None,
                "Max_Pain_Strike": None
            })
            continue

        df = pd.DataFrame(rows).sort_values("Strike").reset_index(drop=True)
        
        # Save option chain CSV
        out_path = os.path.join(out_dir, f"{symbol}_{expiry}.csv".replace("/", "-"))
        try:
            df.to_csv(out_path, index=False)
        except Exception as e:
            if VERBOSE_ERRORS:
                print(f"[WARN] Error writing {out_path}: {e}")

        # Calculate PCR
        tcoi, tpoi, pcr = compute_pcr(rows)
        
        # Calculate Max Pain
        max_pain_result = calculate_max_pain(df)
        max_pain_strike = max_pain_result['max_pain_strike'] if max_pain_result else None
        
        # Generate Max Pain plot if enabled
        if ENABLE_PLOTS and max_pain_result:
            plot_path = os.path.join(out_dir, f"{symbol}_{expiry}_MaxPain.png".replace("/", "-"))
            plot_title = f"{symbol} - {expiry} Max Pain Analysis"
            plot_max_pain(df, plot_title, plot_path)
        
        pcr_maxpain_rows.append({
            "Expiry": expiry,
            "Total_Call_OI": tcoi,
            "Total_Put_OI": tpoi,
            "PCR": pcr,
            "Max_Pain_Strike": max_pain_strike
        })

    # Save master summary file for index
    master_df = pd.DataFrame(pcr_maxpain_rows, 
                            columns=["Expiry", "Total_Call_OI", "Total_Put_OI", "PCR", "Max_Pain_Strike"])
    master_name = f"{symbol}_Analysis_Summary.csv"
    master_path = os.path.join(out_dir, master_name)
    try:
        master_df.sort_values("Expiry", inplace=True)
        master_df.to_csv(master_path, index=False)
        print(f"  ✓ {symbol} summary saved ({len(master_df)} expiries)")
    except Exception as e:
        print(f"  [ERROR] Writing index master file {master_path}: {e}")

    return pcr_maxpain_rows

# ==========================
# MAIN EXECUTION
# ==========================
def main():
    """Main execution flow"""
    print("=" * 70)
    print("NSE OPTIONS ANALYZER - PCR & MAX PAIN CALCULATOR")
    print("=" * 70)
    
    ensure_dir(BASE_DIR)
    print(f"\n📁 Output directory: {os.path.abspath(BASE_DIR)}\n")

    equities_out_base = BASE_DIR
    indices_base = os.path.join(BASE_DIR, "index_options")
    ensure_dir(indices_base)

    # Initialize session
    session = new_session()
    if not warm_cookies(session):
        print("[WARN] Warmup did not return 200 — proceeding anyway...\n")

    # ==========================================
    # STEP 1: Determine target expiries from RELIANCE
    # ==========================================
    print("STEP 1: Determining target expiries from RELIANCE...")
    try:
        base_symbol = "RELIANCE"
        reliance_data = fetch_option_chain(session, base_symbol, is_index=False)
        
        if reliance_data is None:
            print("[ERROR] RELIANCE has no option data. Using fallback...")
            # Fallback: Try another liquid stock
            for fallback in ["SBIN", "HDFCBANK", "INFY", "TCS"]:
                print(f"  Trying {fallback}...")
                reliance_data = fetch_option_chain(session, fallback, is_index=False)
                if reliance_data:
                    base_symbol = fallback
                    break
        
        if reliance_data is None:
            raise RuntimeError("Cannot determine target expiries from any fallback symbol")
        
        target_expiries = reliance_data.get("records", {}).get("expiryDates", [])
        if len(target_expiries) < 3:
            raise RuntimeError(f"Could not find 3 expiries for {base_symbol}. Found: {target_expiries}")
        target_expiries = target_expiries[:3]
        print(f"✓ Using {base_symbol} expiries: {target_expiries}\n")
    except Exception as e:
        print(f"[FATAL] Cannot determine target expiries: {e}")
        if VERBOSE_ERRORS:
            traceback.print_exc()
        return

    # ==========================================
    # STEP 2: Process all equities (multithreaded)
    # ==========================================
    print("=" * 70)
    print("STEP 2: Processing Equities (PCR + Max Pain)...")
    print("=" * 70)
    print("Note: Symbols without options data will be skipped silently\n")
    
    max_workers = 6  # Reduced to avoid rate limiting
    futures = []
    results = {exp: [] for exp in target_expiries}
    skipped_count = 0
    processed_count = 0
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for raw in RAW_TICKERS:
            futures.append(executor.submit(process_symbol_threadsafe, raw, target_expiries, equities_out_base))
        
        completed = 0
        total = len(futures)
        for fut in as_completed(futures):
            completed += 1
            try:
                sym_results = fut.result()
                if sym_results:
                    processed_count += 1
                    for row in sym_results:
                        exp = row.get("Expiry")
                        if exp in results:
                            results[exp].append(row)
                else:
                    skipped_count += 1
            except Exception as e:
                skipped_count += 1
                if VERBOSE_ERRORS:
                    print(f"\n[ERROR] Worker raised: {e}")
                    traceback.print_exc()
            
            # Progress indicator
            print(f"  Progress: {completed}/{total} | ✓ Processed: {processed_count} | ⊘ Skipped: {skipped_count}", end='\r')
    
    print(f"\n\n✓ Completed: {processed_count} symbols with options data")
    print(f"⊘ Skipped: {skipped_count} symbols (no options data or errors)\n")

    # Write master PCR + Max Pain files for equities (3 files)
    print("Generating master equity summary files...")
    for exp in target_expiries:
        rows = results.get(exp, [])
        if not rows:
            print(f"  ⚠ No data for expiry {exp}")
            continue
            
        df = pd.DataFrame(rows, columns=["Symbol", "Expiry", "Total_Call_OI", "Total_Put_OI", "PCR", "Max_Pain_Strike"])
        df.sort_values("Symbol", inplace=True)
        master_name = f"Equities_Analysis_{exp}.csv".replace("/", "-")
        master_path = os.path.join(BASE_DIR, master_name)
        try:
            df.to_csv(master_path, index=False)
            print(f"  ✓ {master_name} ({len(df)} symbols)")
        except Exception as e:
            print(f"  [ERROR] Writing {master_path}: {e}")

    # ==========================================
    # STEP 3: Process indices (all expiries)
    # ==========================================
    print("\n" + "=" * 70)
    print("STEP 3: Processing Indices (PCR + Max Pain)...")
    print("=" * 70 + "\n")
    
    index_success = 0
    index_failed = 0
    
    for idx in INDEX_LIST:
        try:
            print(f"📊 Processing {idx}...")
            s = new_session()
            warm_cookies(s)
            process_index_all_expiries(s, idx, indices_base)
            index_success += 1
        except Exception as e:
            print(f"  [ERROR] Failed to process {idx}: {e}")
            index_failed += 1
            if VERBOSE_ERRORS:
                traceback.print_exc()

    # ==========================================
    # FINAL SUMMARY
    # ==========================================
    print("\n" + "=" * 70)
    print("✅ ANALYSIS COMPLETE!")
    print("=" * 70)
    print(f"\n📂 All results saved to: {os.path.abspath(BASE_DIR)}")
    print("\n📊 Summary:")
    print(f"  • Equities processed: {processed_count}/{len(RAW_TICKERS)}")
    print(f"  • Equities skipped: {skipped_count} (no options data)")
    print(f"  • Indices processed: {index_success}/{len(INDEX_LIST)}")
    if index_failed > 0:
        print(f"  • Indices failed: {index_failed}")
    print(f"  • Master equity files: {len(target_expiries)} expiries")
    print(f"  • Target expiries: {', '.join(target_expiries)}")
    
    print("\n📋 Generated Files:")
    print("  ├─ Equities_Analysis_<expiry>.csv (master PCR + Max Pain)")
    print("  ├─ <SYMBOL>/ (individual equity folders)")
    print("  │   └─ <SYMBOL>_<expiry>.csv (option chain data)")
    print("  └─ index_options/")
    print("      ├─ NIFTY/")
    print("      │   ├─ NIFTY_<expiry>.csv (all expiries)")
    print("      │   └─ NIFTY_Analysis_Summary.csv")
    print("      └─ BANKNIFTY/")
    print("          ├─ BANKNIFTY_<expiry>.csv (all expiries)")
    print("          └─ BANKNIFTY_Analysis_Summary.csv")
    
    print("\n💡 Each file contains:")
    print("  ✓ Option chain data (Strike, OI, LTP, IV, Bid/Ask)")
    print("  ✓ PCR (Put-Call Ratio) = Put OI / Call OI")
    print("  ✓ Max Pain Strike (minimum loss for option writers)")
    
    if ENABLE_PLOTS:
        print("  ✓ Max Pain visualization plots (.png)")
    
    print("\n" + "=" * 70)
    
    # Save processing log
    log_path = os.path.join(BASE_DIR, "processing_log.txt")
    try:
        with open(log_path, 'w') as f:
            f.write("NSE Options Analysis - Processing Log\n")
            f.write("=" * 50 + "\n\n")
            f.write(f"Total tickers attempted: {len(RAW_TICKERS)}\n")
            f.write(f"Successfully processed: {processed_count}\n")
            f.write(f"Skipped (no options): {skipped_count}\n")
            f.write(f"Target expiries: {', '.join(target_expiries)}\n")
            f.write(f"\nIndices processed: {index_success}/{len(INDEX_LIST)}\n")
            if index_failed > 0:
                f.write(f"Indices failed: {index_failed}\n")
        print(f"📄 Processing log saved: processing_log.txt")
    except Exception as e:
        if VERBOSE_ERRORS:
            print(f"[WARN] Could not save log: {e}")

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n\n⚠ Process interrupted by user")
        sys.exit(0)
    except Exception as e:
        print(f"\n[FATAL] Unhandled exception in main(): {e}")
        traceback.print_exc()
        sys.exit(1)

NSE OPTIONS ANALYZER - PCR & MAX PAIN CALCULATOR

📁 Output directory: C:\Users\sarda\Desktop\nse_options_analysis

STEP 1: Determining target expiries from RELIANCE...
✓ Using RELIANCE expiries: ['25-Nov-2025', '30-Dec-2025', '27-Jan-2026']

STEP 2: Processing Equities (PCR + Max Pain)...
Note: Symbols without options data will be skipped silently

  Progress: 215/215 | ✓ Processed: 205 | ⊘ Skipped: 10

✓ Completed: 205 symbols with options data
⊘ Skipped: 10 symbols (no options data or errors)

Generating master equity summary files...
  ✓ Equities_Analysis_25-Nov-2025.csv (205 symbols)
  ✓ Equities_Analysis_30-Dec-2025.csv (204 symbols)
  ✓ Equities_Analysis_27-Jan-2026.csv (197 symbols)

STEP 3: Processing Indices (PCR + Max Pain)...

📊 Processing NIFTY...
  ✓ NIFTY summary saved (18 expiries)
📊 Processing BANKNIFTY...
  ✓ BANKNIFTY summary saved (6 expiries)

✅ ANALYSIS COMPLETE!

📂 All results saved to: C:\Users\sarda\Desktop\nse_options_analysis

📊 Summary:
  • Equities processed

In [3]:
#!/usr/bin/env python3
"""
NSE Options Advanced Analyzer & Signal Generator (Jupyter Compatible)
FIXED VERSION - All warnings and errors resolved

Run this in Cell 2 after your data collection script.

Fixes:
1. SettingWithCopyWarning resolved (df_clean['distance'])
2. TypeError in index analysis resolved (None values)
3. Improved error handling
4. Better data validation
"""

import os
import sys
from datetime import datetime
from typing import Dict, List, Optional, Tuple
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

# Suppress pandas warnings for cleaner output
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

# ==========================
# CONFIGURATION
# ==========================
BASE_DIR = os.path.join(os.path.expanduser("~"), "Desktop", "nse_options_analysis")
OUTPUT_DIR = os.path.join(os.path.expanduser("~"), "Desktop", "nse_options_analysis_pro")
HISTORICAL_DIR = os.path.join(OUTPUT_DIR, "historical_tracking")
SIGNALS_DIR = os.path.join(OUTPUT_DIR, "trade_signals")
REPORTS_DIR = os.path.join(OUTPUT_DIR, "reports")

# Analysis thresholds
PCR_EXTREME_BULLISH = 1.5
PCR_EXTREME_BEARISH = 0.7
PCR_NEUTRAL_LOW = 0.9
PCR_NEUTRAL_HIGH = 1.1

MAX_PAIN_DISTANCE_THRESHOLD = 5.0
MAX_PAIN_STRONG_THRESHOLD = 8.0

IV_PERCENTILE_HIGH = 75
IV_PERCENTILE_LOW = 25

# ==========================
# HELPER FUNCTIONS
# ==========================
def ensure_dir(path):
    """Create directory if it doesn't exist"""
    os.makedirs(path, exist_ok=True)

def get_today_str():
    """Get today's date as string"""
    return datetime.now().strftime("%Y-%m-%d")

def safe_float(val, default=None):
    """Safely convert to float"""
    try:
        if val is None or pd.isna(val):
            return default
        return float(val)
    except:
        return default

# ==========================
# DATA LOADING
# ==========================
def load_equity_analysis(expiry: str) -> Optional[pd.DataFrame]:
    """Load equity analysis CSV for a specific expiry"""
    file_path = os.path.join(BASE_DIR, f"Equities_Analysis_{expiry}.csv".replace("/", "-"))
    if not os.path.exists(file_path):
        print(f"⚠ File not found: {file_path}")
        return None
    return pd.read_csv(file_path)

def load_index_analysis(index_name: str) -> Optional[pd.DataFrame]:
    """Load index analysis summary CSV"""
    file_path = os.path.join(BASE_DIR, "index_options", index_name, f"{index_name}_Analysis_Summary.csv")
    if not os.path.exists(file_path):
        print(f"⚠ File not found: {file_path}")
        return None
    return pd.read_csv(file_path)

def load_option_chain(symbol: str, expiry: str, is_index=False) -> Optional[pd.DataFrame]:
    """Load detailed option chain for a symbol"""
    if is_index:
        file_path = os.path.join(BASE_DIR, "index_options", symbol, f"{symbol}_{expiry}.csv".replace("/", "-"))
    else:
        file_path = os.path.join(BASE_DIR, symbol, f"{symbol}_{expiry}.csv".replace("/", "-"))
    
    if not os.path.exists(file_path):
        return None
    return pd.read_csv(file_path)

def get_underlying_price(df: pd.DataFrame) -> Optional[float]:
    """Extract underlying price (CMP) from option chain"""
    if 'UnderlyingValue' in df.columns:
        price = df['UnderlyingValue'].dropna().iloc[0] if not df['UnderlyingValue'].dropna().empty else None
        return safe_float(price)
    return None

# ==========================
# OI ANALYSIS
# ==========================
def analyze_oi_levels(df: pd.DataFrame) -> Dict:
    """Analyze OI distribution to find support/resistance levels"""
    if df.empty:
        return {}
    
    max_call_oi_strike = df.loc[df['Call_OI'].idxmax(), 'Strike'] if df['Call_OI'].sum() > 0 else None
    max_put_oi_strike = df.loc[df['Put_OI'].idxmax(), 'Strike'] if df['Put_OI'].sum() > 0 else None
    
    top_call_strikes = df.nlargest(3, 'Call_OI')[['Strike', 'Call_OI']].to_dict('records')
    top_put_strikes = df.nlargest(3, 'Put_OI')[['Strike', 'Put_OI']].to_dict('records')
    
    avg_call_oi = df['Call_OI'].mean()
    avg_put_oi = df['Put_OI'].mean()
    
    strong_resistance = df[df['Call_OI'] > 2 * avg_call_oi]['Strike'].tolist()
    strong_support = df[df['Put_OI'] > 2 * avg_put_oi]['Strike'].tolist()
    
    return {
        'max_call_oi_strike': max_call_oi_strike,
        'max_put_oi_strike': max_put_oi_strike,
        'top_call_strikes': top_call_strikes,
        'top_put_strikes': top_put_strikes,
        'strong_resistance_zones': strong_resistance,
        'strong_support_zones': strong_support,
        'total_call_oi': df['Call_OI'].sum(),
        'total_put_oi': df['Put_OI'].sum()
    }

# ==========================
# IV ANALYSIS (FIXED)
# ==========================
def analyze_iv(df: pd.DataFrame) -> Dict:
    """Analyze Implied Volatility patterns - FIXED VERSION"""
    if df.empty or 'CE_IV' not in df.columns or 'PE_IV' not in df.columns:
        return {}
    
    # FIX: Create explicit copy to avoid SettingWithCopyWarning
    df_clean = df.dropna(subset=['CE_IV', 'PE_IV']).copy()
    if df_clean.empty:
        return {}
    
    avg_call_iv = df_clean['CE_IV'].mean()
    avg_put_iv = df_clean['PE_IV'].mean()
    iv_skew = avg_put_iv - avg_call_iv
    
    # Find ATM IV
    underlying = get_underlying_price(df)
    if underlying:
        # FIX: Use .loc to properly assign values
        df_clean.loc[:, 'distance'] = abs(df_clean['Strike'] - underlying)
        atm_row = df_clean.loc[df_clean['distance'].idxmin()]
        atm_call_iv = safe_float(atm_row['CE_IV'])
        atm_put_iv = safe_float(atm_row['PE_IV'])
    else:
        atm_call_iv = None
        atm_put_iv = None
    
    return {
        'avg_call_iv': avg_call_iv,
        'avg_put_iv': avg_put_iv,
        'iv_skew': iv_skew,
        'atm_call_iv': atm_call_iv,
        'atm_put_iv': atm_put_iv,
        'max_call_iv': df_clean['CE_IV'].max(),
        'min_call_iv': df_clean['CE_IV'].min(),
        'max_put_iv': df_clean['PE_IV'].max(),
        'min_put_iv': df_clean['PE_IV'].min()
    }

# ==========================
# SIGNAL GENERATION
# ==========================
def calculate_max_pain_signal(cmp: float, max_pain: float, pcr: float) -> Dict:
    """Generate signal based on CMP vs Max Pain divergence"""
    if not cmp or not max_pain or not pcr:
        return {'signal': 'NEUTRAL', 'strength': 0, 'reason': 'Insufficient data', 'distance_pct': 0}
    
    distance_pct = ((max_pain - cmp) / cmp) * 100
    abs_distance = abs(distance_pct)
    
    if abs_distance < MAX_PAIN_DISTANCE_THRESHOLD:
        strength = 1
    elif abs_distance < MAX_PAIN_STRONG_THRESHOLD:
        strength = 2
    else:
        strength = 3
    
    if distance_pct > 0:
        if pcr > 1.0:
            signal = 'BULLISH'
            reason = f'CMP below Max Pain ({distance_pct:.2f}%), PCR={pcr:.2f} (bearish sentiment → contrarian bullish)'
        else:
            signal = 'WEAK_BULLISH'
            reason = f'CMP below Max Pain ({distance_pct:.2f}%) but PCR={pcr:.2f} (not confirming)'
    else:
        if pcr < 1.0:
            signal = 'BEARISH'
            reason = f'CMP above Max Pain ({abs(distance_pct):.2f}%), PCR={pcr:.2f} (bullish sentiment → contrarian bearish)'
        else:
            signal = 'WEAK_BEARISH'
            reason = f'CMP above Max Pain ({abs(distance_pct):.2f}%) but PCR={pcr:.2f} (not confirming)'
    
    return {
        'signal': signal,
        'strength': strength,
        'distance_pct': distance_pct,
        'reason': reason
    }

def calculate_pcr_signal(pcr: float) -> Dict:
    """Generate signal based on PCR extremes"""
    if not pcr:
        return {'signal': 'NEUTRAL', 'strength': 0, 'reason': 'No PCR data'}
    
    if pcr >= PCR_EXTREME_BULLISH:
        strength = 3 if pcr > 1.8 else 2
        return {
            'signal': 'BULLISH',
            'strength': strength,
            'reason': f'Extreme bearish sentiment (PCR={pcr:.2f}) → Contrarian bullish'
        }
    elif pcr <= PCR_EXTREME_BEARISH:
        strength = 3 if pcr < 0.5 else 2
        return {
            'signal': 'BEARISH',
            'strength': strength,
            'reason': f'Extreme bullish sentiment (PCR={pcr:.2f}) → Contrarian bearish'
        }
    elif PCR_NEUTRAL_LOW <= pcr <= PCR_NEUTRAL_HIGH:
        return {
            'signal': 'NEUTRAL',
            'strength': 0,
            'reason': f'PCR in neutral zone ({pcr:.2f})'
        }
    elif pcr > PCR_NEUTRAL_HIGH:
        return {
            'signal': 'WEAK_BULLISH',
            'strength': 1,
            'reason': f'Slightly bearish sentiment (PCR={pcr:.2f})'
        }
    else:
        return {
            'signal': 'WEAK_BEARISH',
            'strength': 1,
            'reason': f'Slightly bullish sentiment (PCR={pcr:.2f})'
        }

def calculate_oi_signal(cmp: float, oi_analysis: Dict) -> Dict:
    """Generate signal based on OI distribution"""
    if not cmp or not oi_analysis:
        return {'signal': 'NEUTRAL', 'strength': 0, 'reason': 'No OI data'}
    
    resistance = oi_analysis.get('max_call_oi_strike')
    support = oi_analysis.get('max_put_oi_strike')
    
    if not resistance or not support:
        return {'signal': 'NEUTRAL', 'strength': 0, 'reason': 'Insufficient OI data'}
    
    range_size = resistance - support
    if range_size <= 0:
        return {'signal': 'NEUTRAL', 'strength': 0, 'reason': 'Invalid S/R levels'}
    
    position_pct = ((cmp - support) / range_size) * 100
    
    if position_pct < 25:
        signal = 'BULLISH'
        reason = f'Near support ({support:.0f}), resistance at {resistance:.0f}'
        strength = 2
    elif position_pct > 75:
        signal = 'BEARISH'
        reason = f'Near resistance ({resistance:.0f}), support at {support:.0f}'
        strength = 2
    else:
        signal = 'NEUTRAL'
        reason = f'Mid-range between support {support:.0f} and resistance {resistance:.0f}'
        strength = 0
    
    return {
        'signal': signal,
        'strength': strength,
        'reason': reason,
        'support': support,
        'resistance': resistance,
        'position_pct': position_pct
    }

def calculate_iv_signal(iv_analysis: Dict) -> Dict:
    """Generate trading strategy based on IV levels"""
    if not iv_analysis or 'atm_call_iv' not in iv_analysis:
        return {'signal': 'NEUTRAL', 'strength': 0, 'reason': 'No IV data', 'atm_iv': None, 'iv_skew': 0}
    
    atm_call = iv_analysis.get('atm_call_iv', 0) or 0
    atm_put = iv_analysis.get('atm_put_iv', 0) or 0
    atm_iv = (atm_call + atm_put) / 2 if (atm_call or atm_put) else 0
    iv_skew = iv_analysis.get('iv_skew', 0)
    
    if not atm_iv:
        return {'signal': 'NEUTRAL', 'strength': 0, 'reason': 'No ATM IV data', 'atm_iv': None, 'iv_skew': 0}
    
    if atm_iv > 30:
        signal = 'SELL_OPTIONS'
        strength = 2 if atm_iv > 40 else 1
        reason = f'High IV ({atm_iv:.1f}%) → Favor premium selling'
    elif atm_iv < 15:
        signal = 'BUY_OPTIONS'
        strength = 2 if atm_iv < 10 else 1
        reason = f'Low IV ({atm_iv:.1f}%) → Favor option buying'
    else:
        signal = 'NEUTRAL'
        strength = 0
        reason = f'Moderate IV ({atm_iv:.1f}%)'
    
    skew_note = ''
    if abs(iv_skew) > 3:
        if iv_skew > 0:
            skew_note = ' | Put IV > Call IV (bearish skew)'
        else:
            skew_note = ' | Call IV > Put IV (bullish skew)'
    
    return {
        'signal': signal,
        'strength': strength,
        'reason': reason + skew_note,
        'atm_iv': atm_iv,
        'iv_skew': iv_skew
    }

# ==========================
# COMBINED SIGNAL SCORING
# ==========================
def generate_combined_signal(symbol: str, expiry: str, is_index=False) -> Optional[Dict]:
    """Combine all signals to generate final trade recommendation"""
    df = load_option_chain(symbol, expiry, is_index)
    if df is None or df.empty:
        return None
    
    cmp = get_underlying_price(df)
    if not cmp:
        return None
    
    if is_index:
        summary = load_index_analysis(symbol)
        if summary is None:
            return None
        row = summary[summary['Expiry'] == expiry]
    else:
        summary = load_equity_analysis(expiry)
        if summary is None:
            return None
        row = summary[summary['Symbol'] == symbol]
    
    if row.empty:
        return None
    
    pcr = safe_float(row.iloc[0]['PCR'])
    max_pain = safe_float(row.iloc[0]['Max_Pain_Strike'])
    
    # Validate essential data
    if not pcr or not max_pain:
        return None
    
    max_pain_sig = calculate_max_pain_signal(cmp, max_pain, pcr)
    pcr_sig = calculate_pcr_signal(pcr)
    
    oi_analysis = analyze_oi_levels(df)
    oi_sig = calculate_oi_signal(cmp, oi_analysis)
    
    iv_analysis = analyze_iv(df)
    iv_sig = calculate_iv_signal(iv_analysis)
    
    signal_map = {'BULLISH': 3, 'WEAK_BULLISH': 1, 'NEUTRAL': 0, 'WEAK_BEARISH': -1, 'BEARISH': -3}
    
    max_pain_score = signal_map.get(max_pain_sig['signal'], 0) * max_pain_sig['strength']
    pcr_score = signal_map.get(pcr_sig['signal'], 0) * pcr_sig['strength']
    oi_score = signal_map.get(oi_sig['signal'], 0) * oi_sig['strength']
    
    total_score = max_pain_score + pcr_score + oi_score
    max_possible = 3 * 3 + 3 * 3 + 2 * 3
    
    confidence_pct = (abs(total_score) / max_possible) * 100
    
    if total_score >= 6:
        final_signal = 'STRONG_BULLISH'
    elif total_score >= 3:
        final_signal = 'BULLISH'
    elif total_score >= 1:
        final_signal = 'WEAK_BULLISH'
    elif total_score <= -6:
        final_signal = 'STRONG_BEARISH'
    elif total_score <= -3:
        final_signal = 'BEARISH'
    elif total_score <= -1:
        final_signal = 'WEAK_BEARISH'
    else:
        final_signal = 'NEUTRAL'
    
    return {
        'symbol': symbol,
        'expiry': expiry,
        'cmp': cmp,
        'max_pain': max_pain,
        'pcr': pcr,
        'final_signal': final_signal,
        'confidence_pct': confidence_pct,
        'total_score': total_score,
        'max_pain_signal': max_pain_sig,
        'pcr_signal': pcr_sig,
        'oi_signal': oi_sig,
        'iv_signal': iv_sig,
        'oi_analysis': oi_analysis,
        'iv_analysis': iv_analysis
    }

# ==========================
# STRIKE RECOMMENDATION
# ==========================
def recommend_strikes(signal_data: Dict, df: pd.DataFrame) -> Dict:
    """Recommend specific strikes to trade"""
    if not signal_data or df is None or df.empty:
        return {}
    
    cmp = signal_data['cmp']
    final_signal = signal_data['final_signal']
    iv_signal = signal_data['iv_signal']['signal']
    
    recommendations = []
    df = df.dropna(subset=['Strike', 'Call_LTP', 'Put_LTP'])
    
    if 'BULLISH' in final_signal:
        if iv_signal == 'BUY_OPTIONS':
            atm_calls = df[df['Strike'] >= cmp].nsmallest(3, 'Strike')
            for _, row in atm_calls.iterrows():
                strike = row['Strike']
                premium = safe_float(row['Call_LTP'], 0)
                if premium > 0:
                    risk = premium
                    potential_reward = (strike * 0.05)
                    rr_ratio = potential_reward / risk if risk > 0 else 0
                    recommendations.append({
                        'type': 'BUY_CALL',
                        'strike': strike,
                        'premium': premium,
                        'risk': risk,
                        'potential_reward': potential_reward,
                        'rr_ratio': rr_ratio,
                        'description': f'Buy {strike} Call @ ₹{premium:.2f}'
                    })
        else:
            otm_puts = df[df['Strike'] < cmp].nlargest(3, 'Strike')
            for _, row in otm_puts.iterrows():
                strike = row['Strike']
                premium = safe_float(row['Put_LTP'], 0)
                if premium > 0:
                    risk = strike * 0.1
                    reward = premium
                    rr_ratio = reward / risk if risk > 0 else 0
                    recommendations.append({
                        'type': 'SELL_PUT',
                        'strike': strike,
                        'premium': premium,
                        'reward': reward,
                        'risk': risk,
                        'rr_ratio': rr_ratio,
                        'description': f'Sell {strike} Put @ ₹{premium:.2f}'
                    })
    
    elif 'BEARISH' in final_signal:
        if iv_signal == 'BUY_OPTIONS':
            atm_puts = df[df['Strike'] <= cmp].nlargest(3, 'Strike')
            for _, row in atm_puts.iterrows():
                strike = row['Strike']
                premium = safe_float(row['Put_LTP'], 0)
                if premium > 0:
                    risk = premium
                    potential_reward = (strike * 0.05)
                    rr_ratio = potential_reward / risk if risk > 0 else 0
                    recommendations.append({
                        'type': 'BUY_PUT',
                        'strike': strike,
                        'premium': premium,
                        'risk': risk,
                        'potential_reward': potential_reward,
                        'rr_ratio': rr_ratio,
                        'description': f'Buy {strike} Put @ ₹{premium:.2f}'
                    })
        else:
            otm_calls = df[df['Strike'] > cmp].nsmallest(3, 'Strike')
            for _, row in otm_calls.iterrows():
                strike = row['Strike']
                premium = safe_float(row['Call_LTP'], 0)
                if premium > 0:
                    risk = strike * 0.1
                    reward = premium
                    rr_ratio = reward / risk if risk > 0 else 0
                    recommendations.append({
                        'type': 'SELL_CALL',
                        'strike': strike,
                        'premium': premium,
                        'reward': reward,
                        'risk': risk,
                        'rr_ratio': rr_ratio,
                        'description': f'Sell {strike} Call @ ₹{premium:.2f}'
                    })
    
    recommendations.sort(key=lambda x: x.get('rr_ratio', 0), reverse=True)
    
    return {
        'top_trades': recommendations[:3] if recommendations else [],
        'total_opportunities': len(recommendations)
    }

# ==========================
# HISTORICAL TRACKING
# ==========================
def append_to_historical(symbol: str, expiry: str, signal_data: Dict, is_index=False):
    """Append today's data to historical CSV"""
    if not signal_data:
        return
    
    today = get_today_str()
    
    row = {
        'Date': today,
        'Symbol': symbol,
        'Expiry': expiry,
        'CMP': signal_data['cmp'],
        'Max_Pain': signal_data['max_pain'],
        'PCR': signal_data['pcr'],
        'Final_Signal': signal_data['final_signal'],
        'Confidence_pct': signal_data['confidence_pct'],
        'Max_Pain_Distance_pct': signal_data['max_pain_signal']['distance_pct'],
        'ATM_IV': signal_data['iv_analysis'].get('atm_call_iv'),
        'IV_Skew': signal_data['iv_analysis'].get('iv_skew'),
        'Call_OI_Total': signal_data['oi_analysis'].get('total_call_oi'),
        'Put_OI_Total': signal_data['oi_analysis'].get('total_put_oi'),
        'Support': signal_data['oi_signal'].get('support'),
        'Resistance': signal_data['oi_signal'].get('resistance')
    }
    
    hist_file = os.path.join(HISTORICAL_DIR, f"{symbol}_history.csv")
    
    if os.path.exists(hist_file):
        hist_df = pd.read_csv(hist_file)
        if not ((hist_df['Date'] == today) & (hist_df['Expiry'] == expiry)).any():
            hist_df = pd.concat([hist_df, pd.DataFrame([row])], ignore_index=True)
            hist_df.to_csv(hist_file, index=False)
    else:
        pd.DataFrame([row]).to_csv(hist_file, index=False)

# ==========================
# MAIN ANALYSIS FUNCTIONS
# ==========================
def generate_market_report(expiry: str):
    """Generate comprehensive market report"""
    print(f"\n{'='*70}")
    print(f"MARKET ANALYSIS REPORT - {expiry}")
    print(f"{'='*70}\n")
    
    summary = load_equity_analysis(expiry)
    if summary is None:
        print("❌ No data available for this expiry")
        return
    
    all_signals = []
    
    print(f"Analyzing {len(summary)} stocks...\n")
    
    for idx, row in summary.iterrows():
        symbol = row['Symbol']
        signal_data = generate_combined_signal(symbol, expiry, is_index=False)
        
        if signal_data:
            all_signals.append(signal_data)
            append_to_historical(symbol, expiry, signal_data, is_index=False)
        
        if (idx + 1) % 10 == 0:
            print(f"  Analyzed: {idx + 1}/{len(summary)}", end='\r')
    
    print(f"\n")
    
    signals_df = pd.DataFrame([{
        'Symbol': s['symbol'],
        'CMP': s['cmp'],
        'Max_Pain': s['max_pain'],
        'PCR': s['pcr'],
        'Signal': s['final_signal'],
        'Confidence': f"{s['confidence_pct']:.1f}%",
        'Score': s['total_score']
    } for s in all_signals])
    
    report_file = os.path.join(REPORTS_DIR, f"Market_Report_{expiry}_{get_today_str()}.csv".replace("/", "-"))
    signals_df.to_csv(report_file, index=False)
    
    print(f"📊 MARKET SUMMARY:")
    print(f"Total stocks analyzed: {len(signals_df)}\n")
    
    signal_counts = signals_df['Signal'].value_counts()
    for sig, count in signal_counts.items():
        print(f"  {sig}: {count}")
    
    print(f"\n🎯 TOP BULLISH OPPORTUNITIES:")
    bullish = signals_df[signals_df['Signal'].str.contains('BULLISH')].nlargest(10, 'Score')
    print(bullish[['Symbol', 'CMP', 'Max_Pain', 'PCR', 'Signal', 'Confidence']].to_string(index=False))
    
    print(f"\n🎯 TOP BEARISH OPPORTUNITIES:")
    bearish = signals_df[signals_df['Signal'].str.contains('BEARISH')].nsmallest(10, 'Score')
    print(bearish[['Symbol', 'CMP', 'Max_Pain', 'PCR', 'Signal', 'Confidence']].to_string(index=False))
    
    print(f"\n💾 Full report saved: {report_file}\n")
    
    return signals_df

def analyze_stock(symbol: str, expiry: str):
    """Detailed analysis for a single stock"""
    print(f"\n{'='*70}")
    print(f"DETAILED ANALYSIS: {symbol} - {expiry}")
    print(f"{'='*70}\n")
    
    is_index = symbol in ['NIFTY', 'BANKNIFTY']
    signal_data = generate_combined_signal(symbol, expiry, is_index)
    
    if not signal_data:
        print("❌ No data available")
        return
    
    df = load_option_chain(symbol, expiry, is_index)
    
    # FIX: Safely format with None checks
    cmp = signal_data['cmp'] or 0
    max_pain = signal_data['max_pain'] or 0
    pcr = signal_data['pcr'] or 0
    distance = signal_data['max_pain_signal']['distance_pct'] or 0
    
    print(f"📍 CMP: ₹{cmp:.2f}")
    print(f"🎯 Max Pain: ₹{max_pain:.2f}")
    print(f"📊 PCR: {pcr:.4f}")
    print(f"📈 Distance from Max Pain: {distance:.2f}%\n")
    
    print(f"{'─'*70}")
    print("SIGNAL BREAKDOWN:")
    print(f"{'─'*70}\n")
    
    print(f"1️⃣ MAX PAIN: {signal_data['max_pain_signal']['signal']} ({'⭐' * signal_data['max_pain_signal']['strength']})")
    print(f"   {signal_data['max_pain_signal']['reason']}\n")
    
    print(f"2️⃣ PCR: {signal_data['pcr_signal']['signal']} ({'⭐' * signal_data['pcr_signal']['strength']})")
    print(f"   {signal_data['pcr_signal']['reason']}\n")
    
    print(f"3️⃣ OI: {signal_data['oi_signal']['signal']} ({'⭐' * signal_data['oi_signal']['strength']})")
    print(f"   {signal_data['oi_signal']['reason']}")
    if 'support' in signal_data['oi_signal']:
        support = signal_data['oi_signal'].get('support', 0) or 0
        resistance = signal_data['oi_signal'].get('resistance', 0) or 0
        print(f"   Support: ₹{support:.2f} | Resistance: ₹{resistance:.2f}\n")
    
    print(f"4️⃣ IV: {signal_data['iv_signal']['signal']} ({'⭐' * signal_data['iv_signal']['strength']})")
    print(f"   {signal_data['iv_signal']['reason']}")
    if signal_data['iv_signal'].get('atm_iv'):
        atm_iv = signal_data['iv_signal']['atm_iv'] or 0
        print(f"   ATM IV: {atm_iv:.2f}%\n")
    
    print(f"{'─'*70}")
    print(f"🎯 FINAL SIGNAL: {signal_data['final_signal']}")
    print(f"📊 Confidence: {signal_data['confidence_pct']:.1f}%")
    print(f"💯 Composite Score: {signal_data['total_score']}")
    print(f"{'─'*70}\n")
    
    # Strike recommendations
    if df is not None and not df.empty:
        strikes = recommend_strikes(signal_data, df)
        
        if strikes and strikes.get('top_trades'):
            print(f"{'='*70}")
            print("RECOMMENDED TRADES:")
            print(f"{'='*70}\n")
            
            for i, trade in enumerate(strikes['top_trades'], 1):
                print(f"{i}. {trade['description']}")
                print(f"   Type: {trade['type']}")
                print(f"   Premium: ₹{trade.get('premium', 0):.2f}")
                if 'risk' in trade and 'potential_reward' in trade:
                    print(f"   Risk: ₹{trade['risk']:.2f}")
                    print(f"   Potential Reward: ₹{trade.get('potential_reward', 0):.2f}")
                    print(f"   Risk-Reward: 1:{trade.get('rr_ratio', 0):.2f}")
                print()
    
    append_to_historical(symbol, expiry, signal_data, is_index)
    
    return signal_data

def analyze_indices():
    """Analyze NIFTY and BANKNIFTY - FIXED VERSION"""
    print(f"\n{'='*70}")
    print("INDEX ANALYSIS")
    print(f"{'='*70}\n")
    
    results = {}
    
    for index in ['NIFTY', 'BANKNIFTY']:
        summary = load_index_analysis(index)
        if summary is None:
            print(f"❌ No data for {index}\n")
            continue
        
        print(f"{'─'*70}")
        print(f"{index}")
        print(f"{'─'*70}\n")
        
        index_results = []
        
        for _, row in summary.head(3).iterrows():
            expiry = row['Expiry']
            signal_data = generate_combined_signal(index, expiry, is_index=True)
            
            if signal_data:
                # FIX: Safely handle None values
                cmp = signal_data.get('cmp') or 0
                max_pain = signal_data.get('max_pain') or 0
                pcr = signal_data.get('pcr') or 0
                confidence = signal_data.get('confidence_pct', 0)
                
                print(f"📅 {expiry}")
                print(f"   CMP: ₹{cmp:.2f} | Max Pain: ₹{max_pain:.2f} | PCR: {pcr:.4f}")
                print(f"   Signal: {signal_data['final_signal']} ({confidence:.1f}% confidence)")
                print(f"   {signal_data['max_pain_signal']['reason']}\n")
                
                append_to_historical(index, expiry, signal_data, is_index=True)
                index_results.append(signal_data)
        
        results[index] = index_results
    
    return results

# ==========================
# INITIALIZE DIRECTORIES
# ==========================
print("Initializing analysis directories...")
ensure_dir(OUTPUT_DIR)
ensure_dir(HISTORICAL_DIR)
ensure_dir(SIGNALS_DIR)
ensure_dir(REPORTS_DIR)

if not os.path.exists(BASE_DIR):
    print(f"\n❌ Error: Base data not found at {BASE_DIR}")
    print("Please run the data collection script first!")
else:
    print(f"✅ Data found at: {BASE_DIR}")
    print(f"✅ Output will be saved to: {OUTPUT_DIR}")
    
    equity_files = [f for f in os.listdir(BASE_DIR) if f.startswith('Equities_Analysis_')]
    if equity_files:
        expiries = [f.replace('Equities_Analysis_', '').replace('.csv', '') for f in equity_files]
        print(f"\n📅 Available expiries: {', '.join(expiries)}")
        print("\n" + "="*70)
        print("✅ READY TO ANALYZE!")
        print("="*70)
        print("\n📚 Available functions:")
        print("  • generate_market_report(expiry) - Full market analysis")
        print("  • analyze_stock(symbol, expiry) - Detailed stock analysis")
        print("  • analyze_indices() - NIFTY & BANKNIFTY analysis")
        print("\n💡 Example usage:")
        print(f"  df = generate_market_report('{expiries[0]}')")
        print(f"  analyze_stock('RELIANCE', '{expiries[0]}')")
        print("  analyze_indices()")
        print("\n" + "="*70 + "\n")
    else:
        print("\n❌ No equity analysis files found!")
        print("Please run the data collection script first!")

Initializing analysis directories...
✅ Data found at: C:\Users\sarda\Desktop\nse_options_analysis
✅ Output will be saved to: C:\Users\sarda\Desktop\nse_options_analysis_pro

📅 Available expiries: 25-Nov-2025, 27-Jan-2026, 30-Dec-2025

✅ READY TO ANALYZE!

📚 Available functions:
  • generate_market_report(expiry) - Full market analysis
  • analyze_stock(symbol, expiry) - Detailed stock analysis
  • analyze_indices() - NIFTY & BANKNIFTY analysis

💡 Example usage:
  df = generate_market_report('25-Nov-2025')
  analyze_stock('RELIANCE', '25-Nov-2025')
  analyze_indices()




In [5]:
# Cell 3: Quick Market Overview
import os

BASE_DIR = os.path.join(os.path.expanduser("~"), "Desktop", "nse_options_analysis")
equity_files = [f for f in os.listdir(BASE_DIR) if f.startswith('Equities_Analysis_')]
expiries = [f.replace('Equities_Analysis_', '').replace('.csv', '') for f in equity_files]

print(f"📅 Available expiries: {expiries}")

# Analyze nearest expiry
if expiries:
    expiry = expiries[0]
    print(f"\n🔍 Analyzing: {expiry}")
    print("=" * 70)
    market_df = generate_market_report(expiry)
    
    # Display summary statistics
    if market_df is not None and not market_df.empty:
        print("\n📈 QUICK STATS:")
        print(f"Total stocks analyzed: {len(market_df)}")
        print(f"Average PCR: {market_df['PCR'].mean():.4f}")
        print(f"Bullish signals: {len(market_df[market_df['Signal'].str.contains('BULLISH')])}")
        print(f"Bearish signals: {len(market_df[market_df['Signal'].str.contains('BEARISH')])}")
        print(f"Neutral signals: {len(market_df[market_df['Signal'] == 'NEUTRAL'])}")
        
        # High confidence trades (>60%)
        market_df['Confidence_num'] = market_df['Confidence'].str.rstrip('%').astype(float)
        high_conf = market_df[market_df['Confidence_num'] > 60]
        print(f"\n🎯 High confidence trades (>60%): {len(high_conf)}")
        
        if not high_conf.empty:
            print("\nTop 5 High Confidence Opportunities:")
            print(high_conf.nlargest(5, 'Confidence_num')[['Symbol', 'Signal', 'Confidence', 'PCR', 'Score']].to_string(index=False))
else:
    print("❌ No expiries found!")

📅 Available expiries: ['25-Nov-2025', '27-Jan-2026', '30-Dec-2025']

🔍 Analyzing: 25-Nov-2025

MARKET ANALYSIS REPORT - 25-Nov-2025

Analyzing 205 stocks...



C:\Users\sarda\AppData\Local\Temp\ipykernel_24240\1148196810.py:554: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hist_df = pd.concat([hist_df, pd.DataFrame([row])], ignore_index=True)


C:\Users\sarda\AppData\Local\Temp\ipykernel_24240\1148196810.py:554: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hist_df = pd.concat([hist_df, pd.DataFrame([row])], ignore_index=True)


C:\Users\sarda\AppData\Local\Temp\ipykernel_24240\1148196810.py:554: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hist_df = pd.concat([hist_df, pd.DataFrame([row])], ignore_index=True)


C:\Users\sarda\AppData\Local\Temp\ipykernel_24240\1148196810.py:554: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hist_df = pd.concat([hist_df, pd.DataFrame([row])], ignore_index=True)


C:\Users\sarda\AppData\Local\Temp\ipykernel_24240\1148196810.py:554: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hist_df = pd.concat([hist_df, pd.DataFrame([row])], ignore_index=True)


C:\Users\sarda\AppData\Local\Temp\ipykernel_24240\1148196810.py:554: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hist_df = pd.concat([hist_df, pd.DataFrame([row])], ignore_index=True)


C:\Users\sarda\AppData\Local\Temp\ipykernel_24240\1148196810.py:554: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hist_df = pd.concat([hist_df, pd.DataFrame([row])], ignore_index=True)


C:\Users\sarda\AppData\Local\Temp\ipykernel_24240\1148196810.py:554: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hist_df = pd.concat([hist_df, pd.DataFrame([row])], ignore_index=True)


  Analyzed: 200/205

📊 MARKET SUMMARY:
Total stocks analyzed: 205

  STRONG_BEARISH: 92
  BEARISH: 52
  WEAK_BEARISH: 31
  WEAK_BULLISH: 19
  NEUTRAL: 8
  STRONG_BULLISH: 2
  BULLISH: 1

🎯 TOP BULLISH OPPORTUNITIES:
   Symbol     CMP  Max_Pain      PCR         Signal Confidence
   CONCOR  522.00     540.0 0.755448 STRONG_BULLISH      25.0%
 EXIDEIND  377.50     390.0 0.705869 STRONG_BULLISH      25.0%
 PETRONET  277.70     290.0 1.123096        BULLISH      16.7%
AMBUJACEM  558.25     570.0 0.673039   WEAK_BULLISH       4.2%
     CAMS 3834.60    3900.0 0.533747   WEAK_BULLISH       4.2%
  CGPOWER  732.00     740.0 0.656610   WEAK_BULLISH       4.2%
   COLPAL 2184.00    2200.0 0.624022   WEAK_BULLISH       4.2%
 GLENMARK 1814.00    1840.0 0.685088   WEAK_BULLISH       4.2%
  HDFCAMC 5481.00    5500.0 0.627669   WEAK_BULLISH       4.2%
      IEX  139.45     145.0 0.581362   WEAK_BULLISH       4.2%

🎯 TOP BEARISH OPPORTUNITIES:
    Symbol     CMP  Max_Pain      PCR         Signal Confiden

In [7]:
# Cell 4: Analyze NIFTY & BANKNIFTY
print("\n" + "=" * 70)
print("📊 INDEX ANALYSIS")
print("=" * 70)

try:
    index_results = analyze_indices()
    
    # Summary of index signals
    print("\n📋 INDEX SUMMARY:")
    for index_name, results in index_results.items():
        if results:
            print(f"\n{index_name}:")
            for result in results:
                expiry = result['expiry']
                signal = result['final_signal']
                conf = result['confidence_pct']
                pcr = result['pcr']
                print(f"  • {expiry}: {signal} (Confidence: {conf:.1f}%, PCR: {pcr:.4f})")
        else:
            print(f"\n{index_name}: No data available")
            
except Exception as e:
    print(f"❌ Error analyzing indices: {e}")
    print("\nTroubleshooting:")
    print("1. Check if index data exists in Desktop/nse_options_analysis/index_options/")
    print("2. Ensure Cell 1 (data collection) completed successfully")
    print("3. Try running Cell 1 again to fetch fresh data")


📊 INDEX ANALYSIS

INDEX ANALYSIS

──────────────────────────────────────────────────────────────────────
NIFTY
──────────────────────────────────────────────────────────────────────

📅 02-Dec-2025
   CMP: ₹25694.95 | Max Pain: ₹25700.00 | PCR: 0.6821
   Signal: BEARISH (20.8% confidence)
   CMP below Max Pain (0.02%) but PCR=0.68 (not confirming)

📅 09-Dec-2025
   CMP: ₹25694.95 | Max Pain: ₹25650.00 | PCR: 1.2064
   Signal: STRONG_BEARISH (25.0% confidence)
   CMP above Max Pain (0.17%) but PCR=1.21 (not confirming)

📅 11-Nov-2025
   CMP: ₹25694.95 | Max Pain: ₹25650.00 | PCR: 1.1074
   Signal: STRONG_BEARISH (25.0% confidence)
   CMP above Max Pain (0.17%) but PCR=1.11 (not confirming)

──────────────────────────────────────────────────────────────────────
BANKNIFTY
──────────────────────────────────────────────────────────────────────

📅 25-Nov-2025
   CMP: ₹58138.15 | Max Pain: ₹58000.00 | PCR: 0.9205
   Signal: BEARISH (12.5% confidence)
   CMP above Max Pain (0.24%), PCR=0.92 (b

C:\Users\sarda\AppData\Local\Temp\ipykernel_24240\1148196810.py:554: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hist_df = pd.concat([hist_df, pd.DataFrame([row])], ignore_index=True)


In [9]:
# Cell 5: Analyze specific stocks with detailed breakdown
stocks_to_analyze = ['RELIANCE', 'INFY', 'HDFCBANK', 'TCS', 'SBIN']
expiry = expiries[0]  # Use nearest expiry

print("\n" + "=" * 70)
print(f"🔍 DETAILED STOCK ANALYSIS - Expiry: {expiry}")
print("=" * 70)

analysis_results = []

for stock in stocks_to_analyze:
    try:
        print(f"\n{'='*70}")
        result = analyze_stock(stock, expiry)
        
        if result:
            analysis_results.append({
                'Symbol': stock,
                'Signal': result['final_signal'],
                'Confidence': f"{result['confidence_pct']:.1f}%",
                'Score': result['total_score'],
                'PCR': result['pcr'],
                'CMP': result['cmp'],
                'Max_Pain': result['max_pain'],
                'Distance_%': result['max_pain_signal']['distance_pct']
            })
        
        print("\n" + "="*70 + "\n")
        
    except Exception as e:
        print(f"⚠ Could not analyze {stock}: {e}\n")

# Create comparison DataFrame
if analysis_results:
    import pandas as pd
    comparison_df = pd.DataFrame(analysis_results)
    
    print("\n" + "=" * 70)
    print("📊 COMPARISON TABLE")
    print("=" * 70)
    print(comparison_df.to_string(index=False))
    
    print("\n💡 INTERPRETATION:")
    print("  • Score > 6: Strong directional signal")
    print("  • Score 3-6: Moderate signal")
    print("  • Score -3 to 3: Weak/Neutral")
    print("  • Confidence > 60%: Reliable signal")
    print("  • PCR > 1.5: Extreme bearishness (contrarian bullish)")
    print("  • PCR < 0.7: Extreme bullishness (contrarian bearish)")


🔍 DETAILED STOCK ANALYSIS - Expiry: 25-Nov-2025


DETAILED ANALYSIS: RELIANCE - 25-Nov-2025

📍 CMP: ₹1493.80
🎯 Max Pain: ₹1500.00
📊 PCR: 0.6263
📈 Distance from Max Pain: 0.42%

──────────────────────────────────────────────────────────────────────
SIGNAL BREAKDOWN:
──────────────────────────────────────────────────────────────────────

1️⃣ MAX PAIN: WEAK_BULLISH (⭐)
   CMP below Max Pain (0.42%) but PCR=0.63 (not confirming)

2️⃣ PCR: BEARISH (⭐⭐)
   Extreme bullish sentiment (PCR=0.63) → Contrarian bearish

3️⃣ OI: NEUTRAL ()
   Invalid S/R levels
4️⃣ IV: NEUTRAL ()
   Moderate IV (17.8%) | Put IV > Call IV (bearish skew)
   ATM IV: 17.80%

──────────────────────────────────────────────────────────────────────
🎯 FINAL SIGNAL: BEARISH
📊 Confidence: 20.8%
💯 Composite Score: -5
──────────────────────────────────────────────────────────────────────

RECOMMENDED TRADES:

1. Sell 1500 Call @ ₹21.30
   Type: SELL_CALL
   Premium: ₹21.30

2. Sell 1510 Call @ ₹15.90
   Type: SELL_CALL
   Prem

In [11]:
#!/usr/bin/env python3
"""
CELL 6: Daily Historical Data Builder & Appender

Run this EVERY DAY after market close (post 3:30 PM IST)

Features:
1. Collects today's data (PCR, Max Pain, OI, IV, Greeks)
2. Appends to master historical database
3. Calculates day-over-day changes
4. Builds patterns database for ML
5. Tracks actual vs predicted outcomes

This is your GOLD MINE - the longer you run this, the better your predictions!
"""

import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json

# ==========================
# CONFIGURATION
# ==========================
BASE_DIR = os.path.join(os.path.expanduser("~"), "Desktop", "nse_options_analysis")
HISTORICAL_DB = os.path.join(os.path.expanduser("~"), "Desktop", "nse_options_historical_db")
MASTER_DB = os.path.join(HISTORICAL_DB, "master_database")
PATTERNS_DB = os.path.join(HISTORICAL_DB, "patterns")
OUTCOMES_DB = os.path.join(HISTORICAL_DB, "outcomes")

# Create directories
for path in [HISTORICAL_DB, MASTER_DB, PATTERNS_DB, OUTCOMES_DB]:
    os.makedirs(path, exist_ok=True)

TODAY = datetime.now().strftime("%Y-%m-%d")
YESTERDAY = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")

print("=" * 70)
print("📊 DAILY HISTORICAL DATA BUILDER")
print("=" * 70)
print(f"\n📅 Date: {TODAY}")
print(f"💾 Database: {HISTORICAL_DB}\n")

# ==========================
# HELPER FUNCTIONS
# ==========================
def safe_float(val, default=0):
    """Safely convert to float"""
    try:
        if val is None or pd.isna(val):
            return default
        return float(val)
    except:
        return default

def calculate_greeks_proxy(row, df, cmp):
    """
    Calculate approximate Greeks using Black-Scholes approximations
    Note: These are PROXIES - not exact, but good enough for analysis
    """
    strike = row['Strike']
    call_ltp = safe_float(row.get('Call_LTP'))
    put_ltp = safe_float(row.get('Put_LTP'))
    
    # Delta proxy (simplified)
    # Call delta: 0 (deep OTM) to 1 (deep ITM)
    # Put delta: -1 (deep ITM) to 0 (deep OTM)
    moneyness = (cmp - strike) / cmp
    
    if call_ltp > 0:
        # Approximate call delta using simple formula
        call_delta = 0.5 + (moneyness * 2)  # Rough approximation
        call_delta = max(0, min(1, call_delta))  # Bound between 0 and 1
    else:
        call_delta = 0
    
    if put_ltp > 0:
        put_delta = -0.5 - (moneyness * 2)
        put_delta = max(-1, min(0, put_delta))
    else:
        put_delta = 0
    
    # Gamma proxy (highest at ATM)
    # Peaks at ATM, decays as you move away
    distance_from_atm = abs(cmp - strike) / cmp
    gamma = max(0, 0.1 * (1 - distance_from_atm * 10))  # Simplified
    
    # Theta proxy (time decay) - negative for option buyers
    # Higher for ATM options
    theta = -gamma * 0.5  # Simplified relationship
    
    # Vega proxy (IV sensitivity)
    vega = gamma * 10  # Simplified
    
    return {
        'call_delta': call_delta,
        'put_delta': put_delta,
        'gamma': gamma,
        'theta': theta,
        'vega': vega
    }

def calculate_oi_changes(symbol, expiry, current_df):
    """
    Calculate OI changes from yesterday
    This is CRITICAL for detecting fresh positioning
    """
    hist_file = os.path.join(MASTER_DB, f"{symbol}_{expiry}_history.csv".replace("/", "-"))
    
    if not os.path.exists(hist_file):
        # First day - no changes
        current_df['Call_OI_Change'] = 0
        current_df['Put_OI_Change'] = 0
        current_df['Call_OI_Pct_Change'] = 0
        current_df['Put_OI_Pct_Change'] = 0
        return current_df
    
    # Load yesterday's data
    hist_df = pd.read_csv(hist_file)
    yesterday_data = hist_df[hist_df['Date'] == YESTERDAY]
    
    if yesterday_data.empty:
        current_df['Call_OI_Change'] = 0
        current_df['Put_OI_Change'] = 0
        current_df['Call_OI_Pct_Change'] = 0
        current_df['Put_OI_Pct_Change'] = 0
        return current_df
    
    # Merge and calculate changes
    merged = current_df.merge(
        yesterday_data[['Strike', 'Call_OI', 'Put_OI']],
        on='Strike',
        how='left',
        suffixes=('', '_prev')
    )
    
    merged['Call_OI_Change'] = merged['Call_OI'] - merged['Call_OI_prev'].fillna(0)
    merged['Put_OI_Change'] = merged['Put_OI'] - merged['Put_OI_prev'].fillna(0)
    
    merged['Call_OI_Pct_Change'] = (
        (merged['Call_OI_Change'] / merged['Call_OI_prev'].replace(0, 1)) * 100
    ).fillna(0)
    
    merged['Put_OI_Pct_Change'] = (
        (merged['Put_OI_Change'] / merged['Put_OI_prev'].replace(0, 1)) * 100
    ).fillna(0)
    
    return merged.drop(columns=['Call_OI_prev', 'Put_OI_prev'])

def enrich_option_chain_data(symbol, expiry, is_index=False):
    """
    Load option chain and enrich with:
    1. Greeks (proxy)
    2. OI changes
    3. Volume/OI ratio
    4. Skew metrics
    """
    # Load raw data
    df = load_option_chain(symbol, expiry, is_index)
    if df is None or df.empty:
        return None
    
    cmp = get_underlying_price(df)
    if not cmp:
        return None
    
    # Calculate OI changes
    df = calculate_oi_changes(symbol, expiry, df)
    
    # Calculate Greeks for each strike
    greeks_list = []
    for idx, row in df.iterrows():
        greeks = calculate_greeks_proxy(row, df, cmp)
        greeks_list.append(greeks)
    
    greeks_df = pd.DataFrame(greeks_list)
    df = pd.concat([df, greeks_df], axis=1)
    
    # Calculate advanced metrics
    df['moneyness'] = (cmp - df['Strike']) / cmp
    df['is_atm'] = (abs(df['Strike'] - cmp) / cmp) < 0.02  # Within 2% = ATM
    
    # Volume to OI ratio (if volume data available)
    # Note: NSE doesn't give volume in API, so we skip this for now
    
    # Add timestamp
    df['Date'] = TODAY
    df['CMP'] = cmp
    
    return df

def calculate_aggregate_metrics(df, symbol, expiry):
    """
    Calculate portfolio-level metrics:
    1. Total Greeks exposure
    2. Net delta
    3. Gamma exposure (GEX)
    4. Put/Call OI ratio by Greeks
    """
    if df is None or df.empty:
        return {}
    
    # Delta-weighted OI
    df['call_delta_oi'] = df['Call_OI'] * df['call_delta']
    df['put_delta_oi'] = df['Put_OI'] * abs(df['put_delta'])
    
    # Gamma exposure
    df['gamma_exposure'] = (df['Call_OI'] - df['Put_OI']) * df['gamma'] * df['Strike']
    
    # ATM metrics (most important)
    atm_df = df[df['is_atm']]
    
    metrics = {
        'symbol': symbol,
        'expiry': expiry,
        'date': TODAY,
        'total_call_oi': df['Call_OI'].sum(),
        'total_put_oi': df['Put_OI'].sum(),
        'pcr': df['Put_OI'].sum() / df['Call_OI'].sum() if df['Call_OI'].sum() > 0 else None,
        
        # OI Changes
        'call_oi_change': df['Call_OI_Change'].sum(),
        'put_oi_change': df['Put_OI_Change'].sum(),
        'net_oi_change': df['Put_OI_Change'].sum() - df['Call_OI_Change'].sum(),
        
        # Greeks
        'net_delta': df['call_delta_oi'].sum() + df['put_delta_oi'].sum(),
        'total_gamma_exposure': df['gamma_exposure'].sum(),
        'max_gamma_strike': df.loc[df['gamma'].idxmax(), 'Strike'] if not df.empty else None,
        
        # ATM metrics
        'atm_call_oi': atm_df['Call_OI'].sum() if not atm_df.empty else 0,
        'atm_put_oi': atm_df['Put_OI'].sum() if not atm_df.empty else 0,
        'atm_pcr': (atm_df['Put_OI'].sum() / atm_df['Call_OI'].sum()) if not atm_df.empty and atm_df['Call_OI'].sum() > 0 else None,
        
        # IV metrics
        'avg_call_iv': df['CE_IV'].mean() if 'CE_IV' in df.columns else None,
        'avg_put_iv': df['PE_IV'].mean() if 'PE_IV' in df.columns else None,
        'iv_skew': (df['PE_IV'].mean() - df['CE_IV'].mean()) if 'CE_IV' in df.columns else None,
    }
    
    return metrics

def append_to_master_database(df, symbol, expiry):
    """
    Append today's enriched data to master historical database
    """
    if df is None or df.empty:
        return
    
    master_file = os.path.join(MASTER_DB, f"{symbol}_{expiry}_history.csv".replace("/", "-"))
    
    if os.path.exists(master_file):
        # Load existing and append
        existing = pd.read_csv(master_file)
        # Remove today's data if exists (in case of re-run)
        existing = existing[existing['Date'] != TODAY]
        combined = pd.concat([existing, df], ignore_index=True)
        combined.to_csv(master_file, index=False)
    else:
        # First time
        df.to_csv(master_file, index=False)
    
    print(f"  ✓ Appended {symbol} {expiry} to master DB")

def save_aggregate_metrics(metrics, symbol, expiry):
    """
    Save daily aggregate metrics (PCR, Greeks, OI changes)
    """
    if not metrics:
        return
    
    agg_file = os.path.join(MASTER_DB, f"{symbol}_{expiry}_aggregates.csv".replace("/", "-"))
    
    metrics_df = pd.DataFrame([metrics])
    
    if os.path.exists(agg_file):
        existing = pd.read_csv(agg_file)
        # Remove today if exists
        existing = existing[existing['date'] != TODAY]
        combined = pd.concat([existing, metrics_df], ignore_index=True)
        combined.to_csv(agg_file, index=False)
    else:
        metrics_df.to_csv(agg_file, index=False)
    
    print(f"  ✓ Saved aggregate metrics for {symbol} {expiry}")

def detect_patterns(symbol, expiry):
    """
    Detect trading patterns from historical data:
    1. PCR reversals
    2. OI buildup/unwinding
    3. Max Pain movement
    """
    agg_file = os.path.join(MASTER_DB, f"{symbol}_{expiry}_aggregates.csv".replace("/", "-"))
    
    if not os.path.exists(agg_file):
        return None
    
    df = pd.read_csv(agg_file)
    if len(df) < 2:
        return None
    
    # Sort by date
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date')
    
    # Calculate changes
    df['pcr_change'] = df['pcr'].diff()
    df['oi_change_trend'] = df['net_oi_change'].diff()
    
    patterns = {
        'symbol': symbol,
        'expiry': expiry,
        'date': TODAY,
        'pcr_trend': 'rising' if df['pcr_change'].iloc[-1] > 0 else 'falling',
        'pcr_velocity': df['pcr_change'].iloc[-1],
        'oi_trend': 'buildup' if df['net_oi_change'].iloc[-1] > 0 else 'unwinding',
        'days_to_expiry': (pd.to_datetime(expiry) - pd.to_datetime(TODAY)).days,
    }
    
    return patterns

def save_patterns(patterns):
    """Save detected patterns for ML training"""
    if not patterns:
        return
    
    patterns_file = os.path.join(PATTERNS_DB, f"patterns_{TODAY}.csv")
    
    patterns_df = pd.DataFrame([patterns])
    
    if os.path.exists(patterns_file):
        existing = pd.read_csv(patterns_file)
        combined = pd.concat([existing, patterns_df], ignore_index=True)
        combined.to_csv(patterns_file, index=False)
    else:
        patterns_df.to_csv(patterns_file, index=False)

# ==========================
# MAIN COLLECTION PROCESS
# ==========================
def collect_and_store_daily_data():
    """
    Main function: Collect today's data and append to historical DB
    """
    print("\n🔄 Starting daily data collection...\n")
    
    # Get list of symbols and expiries from today's run
    equity_files = [f for f in os.listdir(BASE_DIR) if f.startswith('Equities_Analysis_')]
    if not equity_files:
        print("❌ No data files found. Run Cell 1 first!")
        return
    
    expiries = [f.replace('Equities_Analysis_', '').replace('.csv', '') for f in equity_files]
    
    processed_count = 0
    error_count = 0
    
    # Process equities
    for expiry in expiries:
        print(f"\n📅 Processing expiry: {expiry}")
        
        summary = load_equity_analysis(expiry)
        if summary is None:
            continue
        
        for idx, row in summary.iterrows():
            symbol = row['Symbol']
            
            try:
                # Enrich option chain with Greeks, OI changes, etc.
                enriched_df = enrich_option_chain_data(symbol, expiry, is_index=False)
                
                if enriched_df is not None:
                    # Append to master database
                    append_to_master_database(enriched_df, symbol, expiry)
                    
                    # Calculate and save aggregate metrics
                    metrics = calculate_aggregate_metrics(enriched_df, symbol, expiry)
                    save_aggregate_metrics(metrics, symbol, expiry)
                    
                    # Detect patterns
                    patterns = detect_patterns(symbol, expiry)
                    if patterns:
                        save_patterns(patterns)
                    
                    processed_count += 1
                
            except Exception as e:
                error_count += 1
                if error_count <= 5:  # Only show first 5 errors
                    print(f"  ⚠ Error processing {symbol}: {e}")
            
            # Progress
            if (idx + 1) % 20 == 0:
                print(f"  Progress: {idx + 1}/{len(summary)}", end='\r')
    
    # Process indices
    print(f"\n\n📊 Processing indices...")
    for index in ['NIFTY', 'BANKNIFTY']:
        summary = load_index_analysis(index)
        if summary is None:
            continue
        
        for _, row in summary.head(3).iterrows():  # First 3 expiries
            expiry = row['Expiry']
            
            try:
                enriched_df = enrich_option_chain_data(index, expiry, is_index=True)
                
                if enriched_df is not None:
                    append_to_master_database(enriched_df, index, expiry)
                    metrics = calculate_aggregate_metrics(enriched_df, index, expiry)
                    save_aggregate_metrics(metrics, index, expiry)
                    patterns = detect_patterns(index, expiry)
                    if patterns:
                        save_patterns(patterns)
                    
                    processed_count += 1
                    print(f"  ✓ Processed {index} {expiry}")
                    
            except Exception as e:
                print(f"  ⚠ Error processing {index}: {e}")
    
    # Summary
    print("\n" + "=" * 70)
    print("✅ DAILY COLLECTION COMPLETE")
    print("=" * 70)
    print(f"\n📊 Summary:")
    print(f"  • Successfully processed: {processed_count}")
    print(f"  • Errors: {error_count}")
    print(f"  • Database location: {HISTORICAL_DB}")
    
    # Database statistics
    master_files = [f for f in os.listdir(MASTER_DB) if f.endswith('_history.csv')]
    print(f"  • Total historical files: {len(master_files)}")
    
    # Calculate total days of data
    if master_files:
        sample_file = os.path.join(MASTER_DB, master_files[0])
        sample_df = pd.read_csv(sample_file)
        unique_dates = sample_df['Date'].nunique()
        print(f"  • Days of historical data: {unique_dates}")
        
        if unique_dates >= 5:
            print(f"\n🎉 You now have {unique_dates} days of data!")
            print("   Advanced analysis features will become available:")
            print("   • Historical PCR correlation")
            print("   • Pattern recognition")
            print("   • Backtesting capability")
        else:
            print(f"\n💡 Keep collecting! You need at least 5 days for ML features.")
            print(f"   Currently: {unique_dates}/5 days")
    
    print("\n" + "=" * 70)
    
    return {
        'processed': processed_count,
        'errors': error_count,
        'database': HISTORICAL_DB
    }

# ==========================
# RUN COLLECTION
# ==========================
print("\n⏰ IMPORTANT: Run this cell EVERY DAY after market close!")
print("   Recommended time: After 4:00 PM IST\n")

# Auto-run
result = collect_and_store_daily_data()

print("\n💾 Next Steps:")
print("1. Run this cell daily to build your database")
print("2. After 5+ days, run Cell 7 (Advanced Analysis)")
print("3. After 10+ days, run Cell 8 (ML Predictions)")
print("\n" + "=" * 70)

📊 DAILY HISTORICAL DATA BUILDER

📅 Date: 2025-11-11
💾 Database: C:\Users\sarda\Desktop\nse_options_historical_db


⏰ IMPORTANT: Run this cell EVERY DAY after market close!
   Recommended time: After 4:00 PM IST


🔄 Starting daily data collection...


📅 Processing expiry: 25-Nov-2025
  ✓ Appended 360ONE 25-Nov-2025 to master DB
  ✓ Saved aggregate metrics for 360ONE 25-Nov-2025
  ✓ Appended ABB 25-Nov-2025 to master DB
  ✓ Saved aggregate metrics for ABB 25-Nov-2025
  ✓ Appended ABCAPITAL 25-Nov-2025 to master DB
  ✓ Saved aggregate metrics for ABCAPITAL 25-Nov-2025
  ✓ Appended ADANIENSOL 25-Nov-2025 to master DB
  ✓ Saved aggregate metrics for ADANIENSOL 25-Nov-2025
  ✓ Appended ADANIENT 25-Nov-2025 to master DB
  ✓ Saved aggregate metrics for ADANIENT 25-Nov-2025
  ✓ Appended ADANIGREEN 25-Nov-2025 to master DB
  ✓ Saved aggregate metrics for ADANIGREEN 25-Nov-2025
  ✓ Appended ADANIPORTS 25-Nov-2025 to master DB
  ✓ Saved aggregate metrics for ADANIPORTS 25-Nov-2025
  ✓ Appended ALK

C:\Users\sarda\AppData\Local\Temp\ipykernel_24240\221194803.py:282: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined = pd.concat([existing, metrics_df], ignore_index=True)


  ✓ Appended AUBANK 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for AUBANK 27-Jan-2026
  ✓ Appended AUROPHARMA 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for AUROPHARMA 27-Jan-2026
  ✓ Appended AXISBANK 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for AXISBANK 27-Jan-2026
  ✓ Appended BAJAJ-AUTO 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for BAJAJ-AUTO 27-Jan-2026
  ✓ Appended BAJAJFINSV 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for BAJAJFINSV 27-Jan-2026
  ✓ Appended BAJFINANCE 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for BAJFINANCE 27-Jan-2026
  ✓ Appended BANDHANBNK 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for BANDHANBNK 27-Jan-2026
  ✓ Appended BANKBARODA 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for BANKBARODA 27-Jan-2026
  ✓ Appended BANKINDIA 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for BANKINDIA 27-Jan-2026
  ✓ Appended BDL 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for BDL 

C:\Users\sarda\AppData\Local\Temp\ipykernel_24240\221194803.py:282: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined = pd.concat([existing, metrics_df], ignore_index=True)


  ✓ Appended HDFCAMC 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for HDFCAMC 27-Jan-2026
  ✓ Appended HDFCBANK 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for HDFCBANK 27-Jan-2026
  ✓ Appended HDFCLIFE 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for HDFCLIFE 27-Jan-2026
  ✓ Appended HEROMOTOCO 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for HEROMOTOCO 27-Jan-2026
  ✓ Appended HINDALCO 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for HINDALCO 27-Jan-2026
  ✓ Appended HINDPETRO 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for HINDPETRO 27-Jan-2026


...

  ✓ Saved aggregate metrics for TRENT 27-Jan-2026
  ✓ Appended TVSMOTOR 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for TVSMOTOR 27-Jan-2026
  ✓ Appended ULTRACEMCO 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for ULTRACEMCO 27-Jan-2026
  ✓ Appended UNIONBANK 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for UNIONBANK 27-Jan-2026
  ✓ Appended UNITDSPR 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for UNITDSPR 27-Jan-2026
  ✓ Appended UNOMINDA 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for UNOMINDA 27-Jan-2026
  ✓ Appended UPL 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for UPL 27-Jan-2026
  ✓ Appended VBL 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for VBL 27-Jan-2026
  ✓ Appended VEDL 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for VEDL 27-Jan-2026
  ✓ Appended VOLTAS 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for VOLTAS 27-Jan-2026
  ✓ Appended WIPRO 27-Jan-2026 to master DB
  ✓ Saved aggregate metrics for 

C:\Users\sarda\AppData\Local\Temp\ipykernel_24240\221194803.py:282: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined = pd.concat([existing, metrics_df], ignore_index=True)


  ✓ Appended MAXHEALTH 30-Dec-2025 to master DB
  ✓ Saved aggregate metrics for MAXHEALTH 30-Dec-2025
  ✓ Appended MAZDOCK 30-Dec-2025 to master DB
  ✓ Saved aggregate metrics for MAZDOCK 30-Dec-2025
  ✓ Appended MCX 30-Dec-2025 to master DB
  ✓ Saved aggregate metrics for MCX 30-Dec-2025
  ✓ Appended MFSL 30-Dec-2025 to master DB
  ✓ Saved aggregate metrics for MFSL 30-Dec-2025
  ✓ Appended MOTHERSON 30-Dec-2025 to master DB
  ✓ Saved aggregate metrics for MOTHERSON 30-Dec-2025
  ✓ Appended MPHASIS 30-Dec-2025 to master DB
  ✓ Saved aggregate metrics for MPHASIS 30-Dec-2025
  ✓ Appended MUTHOOTFIN 30-Dec-2025 to master DB
  ✓ Saved aggregate metrics for MUTHOOTFIN 30-Dec-2025
  ✓ Appended NATIONALUM 30-Dec-2025 to master DB
  ✓ Saved aggregate metrics for NATIONALUM 30-Dec-2025
  ✓ Appended NAUKRI 30-Dec-2025 to master DB
  ✓ Saved aggregate metrics for NAUKRI 30-Dec-2025
  ✓ Appended NBCC 30-Dec-2025 to master DB
  ✓ Saved aggregate metrics for NBCC 30-Dec-2025
  ✓ Appended NCC 30-De

C:\Users\sarda\AppData\Local\Temp\ipykernel_24240\221194803.py:282: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined = pd.concat([existing, metrics_df], ignore_index=True)


  ✓ Appended RBLBANK 30-Dec-2025 to master DB
  ✓ Saved aggregate metrics for RBLBANK 30-Dec-2025
  ✓ Appended RECLTD 30-Dec-2025 to master DB
  ✓ Saved aggregate metrics for RECLTD 30-Dec-2025
  ✓ Appended RELIANCE 30-Dec-2025 to master DB
  ✓ Saved aggregate metrics for RELIANCE 30-Dec-2025
  ✓ Appended RVNL 30-Dec-2025 to master DB
  ✓ Saved aggregate metrics for RVNL 30-Dec-2025
  ✓ Appended SAIL 30-Dec-2025 to master DB
  ✓ Saved aggregate metrics for SAIL 30-Dec-2025
  ✓ Appended SBICARD 30-Dec-2025 to master DB
  ✓ Saved aggregate metrics for SBICARD 30-Dec-2025
  ✓ Appended SBILIFE 30-Dec-2025 to master DB
  ✓ Saved aggregate metrics for SBILIFE 30-Dec-2025
  ✓ Appended SBIN 30-Dec-2025 to master DB
  ✓ Saved aggregate metrics for SBIN 30-Dec-2025
  ✓ Appended SHREECEM 30-Dec-2025 to master DB
  ✓ Saved aggregate metrics for SHREECEM 30-Dec-2025
  ✓ Appended SHRIRAMFIN 30-Dec-2025 to master DB
  ✓ Saved aggregate metrics for SHRIRAMFIN 30-Dec-2025
  ✓ Appended SIEMENS 30-Dec-20

In [13]:
#!/usr/bin/env python3
"""
CELL 7: Advanced Historical Analysis Engine

Prerequisites: Need at least 5 days of data (run Cell 6 daily)

Features:
1. Historical PCR correlation with price movements
2. Max Pain accuracy tracker
3. OI change pattern analysis
4. Greeks-based positioning
5. Win rate calculator for signals
6. Best time-to-expiry analysis

This shows you WHAT ACTUALLY WORKS!
"""

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from scipy import stats

# ==========================
# CONFIGURATION
# ==========================
HISTORICAL_DB = os.path.join(os.path.expanduser("~"), "Desktop", "nse_options_historical_db")
MASTER_DB = os.path.join(HISTORICAL_DB, "master_database")
ANALYSIS_OUTPUT = os.path.join(HISTORICAL_DB, "advanced_analysis")

os.makedirs(ANALYSIS_OUTPUT, exist_ok=True)

TODAY = datetime.now().strftime("%Y-%m-%d")

print("=" * 70)
print("🔬 ADVANCED HISTORICAL ANALYSIS ENGINE")
print("=" * 70)
print(f"\n📅 Analysis Date: {TODAY}\n")

# ==========================
# DATA VALIDATION
# ==========================
def check_data_availability():
    """Check if we have enough historical data"""
    if not os.path.exists(MASTER_DB):
        print("❌ No historical database found!")
        print("   Please run Cell 6 to start collecting data.\n")
        return False
    
    agg_files = [f for f in os.listdir(MASTER_DB) if f.endswith('_aggregates.csv')]
    
    if not agg_files:
        print("❌ No aggregate data found!")
        print("   Please run Cell 6 first.\n")
        return False
    
    # Check number of days
    sample_file = os.path.join(MASTER_DB, agg_files[0])
    df = pd.read_csv(sample_file)
    num_days = df['date'].nunique()
    
    print(f"📊 Data Status:")
    print(f"   • Files: {len(agg_files)}")
    print(f"   • Days of data: {num_days}\n")
    
    if num_days < 2:
        print("⚠️  Need at least 2 days for basic analysis")
        print(f"   Current: {num_days}/2 days\n")
        return False
    
    if num_days < 5:
        print(f"💡 You have {num_days} days - basic analysis available")
        print("   Collect 5+ days for full ML features!\n")
    else:
        print(f"🎉 You have {num_days} days - full analysis available!\n")
    
    return True

# ==========================
# 1. PCR CORRELATION ANALYSIS
# ==========================
def analyze_pcr_correlation(symbol, expiry):
    """
    Analyze correlation between PCR and next-day price movement
    
    This tells you: Does high PCR actually lead to bullish moves?
    """
    agg_file = os.path.join(MASTER_DB, f"{symbol}_{expiry}_aggregates.csv".replace("/", "-"))
    
    if not os.path.exists(agg_file):
        return None
    
    df = pd.read_csv(agg_file)
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date')
    
    if len(df) < 3:
        return None
    
    # Get CMP from detailed data
    hist_file = os.path.join(MASTER_DB, f"{symbol}_{expiry}_history.csv".replace("/", "-"))
    if os.path.exists(hist_file):
        hist_df = pd.read_csv(hist_file)
        # Get daily CMP (first strike's CMP since it's same for all)
        daily_cmp = hist_df.groupby('Date')['CMP'].first().reset_index()
        daily_cmp.columns = ['date', 'cmp']
        daily_cmp['date'] = pd.to_datetime(daily_cmp['date'])
        
        # Merge with aggregates
        df = df.merge(daily_cmp, on='date', how='left')
        
        # Calculate next-day price change
        df['next_day_return'] = df['cmp'].shift(-1) / df['cmp'] - 1
        df['next_day_return_pct'] = df['next_day_return'] * 100
        
        # Correlation
        correlation = df[['pcr', 'next_day_return_pct']].corr().iloc[0, 1]
        
        # PCR categories
        df['pcr_category'] = pd.cut(df['pcr'], 
                                      bins=[0, 0.7, 0.9, 1.1, 1.5, 10],
                                      labels=['Extreme Bullish', 'Bullish', 'Neutral', 'Bearish', 'Extreme Bearish'])
        
        # Average return by PCR category
        category_returns = df.groupby('pcr_category')['next_day_return_pct'].agg(['mean', 'count', 'std'])
        
        return {
            'symbol': symbol,
            'expiry': expiry,
            'correlation': correlation,
            'category_returns': category_returns,
            'data': df
        }
    
    return None

def run_pcr_correlation_study():
    """
    Run PCR correlation for all symbols
    Find which stocks' PCR is most predictive
    """
    print("📈 PCR Correlation Study")
    print("─" * 70)
    print("Analyzing: Does PCR predict next-day returns?\n")
    
    agg_files = [f for f in os.listdir(MASTER_DB) if f.endswith('_aggregates.csv')]
    
    results = []
    
    for file in agg_files[:20]:  # Analyze top 20
        parts = file.replace('_aggregates.csv', '').split('_')
        if len(parts) >= 2:
            symbol = parts[0]
            expiry = '_'.join(parts[1:])
            
            result = analyze_pcr_correlation(symbol, expiry)
            if result and not pd.isna(result['correlation']):
                results.append({
                    'Symbol': symbol,
                    'Expiry': expiry,
                    'Correlation': result['correlation'],
                    'Strength': 'Strong' if abs(result['correlation']) > 0.5 else 'Moderate' if abs(result['correlation']) > 0.3 else 'Weak'
                })
    
    if not results:
        print("⚠️  Not enough data yet. Need at least 3 days.\n")
        return None
    
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values('Correlation', key=abs, ascending=False)
    
    print("Top 10 Stocks by PCR Predictive Power:")
    print(results_df.head(10).to_string(index=False))
    
    print("\n💡 Interpretation:")
    print("  • Negative correlation: High PCR → Price goes UP (contrarian works!)")
    print("  • Positive correlation: High PCR → Price goes DOWN (sentiment follows)")
    print("  • |Correlation| > 0.5: Strong predictive power")
    
    # Save
    results_df.to_csv(os.path.join(ANALYSIS_OUTPUT, f"pcr_correlation_{TODAY}.csv"), index=False)
    print(f"\n💾 Saved to: pcr_correlation_{TODAY}.csv\n")
    
    return results_df

# ==========================
# 2. MAX PAIN ACCURACY TRACKER
# ==========================
def analyze_max_pain_accuracy(symbol, expiry):
    """
    Check: How often does price actually gravitate toward Max Pain?
    
    Tracks:
    1. Daily distance from Max Pain
    2. Convergence as expiry approaches
    3. Accuracy on expiry day
    """
    agg_file = os.path.join(MASTER_DB, f"{symbol}_{expiry}_aggregates.csv".replace("/", "-"))
    hist_file = os.path.join(MASTER_DB, f"{symbol}_{expiry}_history.csv".replace("/", "-"))
    
    if not os.path.exists(agg_file) or not os.path.exists(hist_file):
        return None
    
    # Load data
    hist_df = pd.read_csv(hist_file)
    hist_df['Date'] = pd.to_datetime(hist_df['Date'])
    
    # Get CMP and Max Pain per day
    daily_data = []
    for date in hist_df['Date'].unique():
        day_data = hist_df[hist_df['Date'] == date]
        cmp = day_data['CMP'].iloc[0]
        
        # Calculate max pain for that day
        strikes = day_data['Strike'].values
        call_oi = day_data['Call_OI'].values
        put_oi = day_data['Put_OI'].values
        
        total_losses = []
        for expiry_strike in strikes:
            call_loss = ((expiry_strike - strikes).clip(min=0) * call_oi).sum()
            put_loss = ((strikes - expiry_strike).clip(min=0) * put_oi).sum()
            total_losses.append(call_loss + put_loss)
        
        if total_losses:
            max_pain = int(strikes[total_losses.index(min(total_losses))])
            distance_pct = ((max_pain - cmp) / cmp) * 100
            
            daily_data.append({
                'date': date,
                'cmp': cmp,
                'max_pain': max_pain,
                'distance_pct': distance_pct,
                'distance_abs': abs(distance_pct)
            })
    
    if not daily_data:
        return None
    
    df = pd.DataFrame(daily_data)
    df = df.sort_values('date')
    
    # Calculate days to expiry
    expiry_date = pd.to_datetime(expiry)
    df['days_to_expiry'] = (expiry_date - df['date']).dt.days
    
    # Convergence analysis
    if len(df) >= 3:
        # Does distance decrease as expiry approaches?
        convergence_corr = df[['days_to_expiry', 'distance_abs']].corr().iloc[0, 1]
        
        return {
            'symbol': symbol,
            'expiry': expiry,
            'avg_distance': df['distance_abs'].mean(),
            'final_distance': df['distance_abs'].iloc[-1] if len(df) > 0 else None,
            'convergence_correlation': convergence_corr,
            'data': df
        }
    
    return None

def run_max_pain_accuracy_study():
    """
    Check Max Pain accuracy across all symbols
    """
    print("🎯 Max Pain Accuracy Study")
    print("─" * 70)
    print("Analyzing: Does price converge to Max Pain?\n")
    
    agg_files = [f for f in os.listdir(MASTER_DB) if f.endswith('_aggregates.csv')]
    
    results = []
    
    for file in agg_files[:20]:
        parts = file.replace('_aggregates.csv', '').split('_')
        if len(parts) >= 2:
            symbol = parts[0]
            expiry = '_'.join(parts[1:])
            
            result = analyze_max_pain_accuracy(symbol, expiry)
            if result:
                results.append({
                    'Symbol': symbol,
                    'Expiry': expiry,
                    'Avg_Distance_%': result['avg_distance'],
                    'Convergence': 'Yes' if result['convergence_correlation'] > 0.3 else 'No',
                    'Convergence_Score': result['convergence_correlation']
                })
    
    if not results:
        print("⚠️  Not enough data yet.\n")
        return None
    
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values('Avg_Distance_%')
    
    print("Top 10 Stocks by Max Pain Accuracy (Lowest Distance):")
    print(results_df.head(10).to_string(index=False))
    
    print("\n💡 Interpretation:")
    print("  • Lower Avg Distance = More accurate Max Pain")
    print("  • Convergence 'Yes' = Price moves toward Max Pain as expiry nears")
    print("  • Use Max Pain more confidently for these stocks!")
    
    # Save
    results_df.to_csv(os.path.join(ANALYSIS_OUTPUT, f"max_pain_accuracy_{TODAY}.csv"), index=False)
    print(f"\n💾 Saved to: max_pain_accuracy_{TODAY}.csv\n")
    
    return results_df

# ==========================
# 3. OI CHANGE PATTERNS
# ==========================
def analyze_oi_change_patterns(symbol, expiry):
    """
    Analyze OI buildup vs unwinding patterns
    
    Fresh OI buildup = New positions (bullish if puts, bearish if calls)
    OI unwinding = Closing positions (reversal signal)
    """
    hist_file = os.path.join(MASTER_DB, f"{symbol}_{expiry}_history.csv".replace("/", "-"))
    
    if not os.path.exists(hist_file):
        return None
    
    df = pd.read_csv(hist_file)
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Get latest 2 days
    dates = sorted(df['Date'].unique())
    if len(dates) < 2:
        return None
    
    latest = dates[-1]
    previous = dates[-2]
    
    latest_data = df[df['Date'] == latest]
    
    # Identify major OI changes
    significant_call_build = latest_data[latest_data['Call_OI_Change'] > latest_data['Call_OI'] * 0.1]
    significant_put_build = latest_data[latest_data['Put_OI_Change'] > latest_data['Put_OI'] * 0.1]
    
    significant_call_unwind = latest_data[latest_data['Call_OI_Change'] < -latest_data['Call_OI'] * 0.1]
    significant_put_unwind = latest_data[latest_data['Put_OI_Change'] < -latest_data['Put_OI'] * 0.1]
    
    return {
        'symbol': symbol,
        'expiry': expiry,
        'date': latest,
        'call_buildup_strikes': significant_call_build['Strike'].tolist(),
        'put_buildup_strikes': significant_put_build['Strike'].tolist(),
        'call_unwind_strikes': significant_call_unwind['Strike'].tolist(),
        'put_unwind_strikes': significant_put_unwind['Strike'].tolist(),
        'total_call_buildup': significant_call_build['Call_OI_Change'].sum(),
        'total_put_buildup': significant_put_build['Put_OI_Change'].sum(),
    }

def run_oi_pattern_study():
    """
    Identify stocks with significant OI changes
    """
    print("🔄 OI Change Pattern Analysis")
    print("─" * 70)
    print("Detecting: Fresh positioning vs. unwinding\n")
    
    agg_files = [f for f in os.listdir(MASTER_DB) if f.endswith('_aggregates.csv')]
    
    results = []
    
    for file in agg_files[:30]:
        parts = file.replace('_aggregates.csv', '').split('_')
        if len(parts) >= 2:
            symbol = parts[0]
            expiry = '_'.join(parts[1:])
            
            result = analyze_oi_change_patterns(symbol, expiry)
            if result:
                # Determine signal
                net_buildup = result['total_put_buildup'] - result['total_call_buildup']
                
                if net_buildup > 10000:
                    signal = "BULLISH (Put buildup)"
                elif net_buildup < -10000:
                    signal = "BEARISH (Call buildup)"
                else:
                    signal = "NEUTRAL"
                
                results.append({
                    'Symbol': symbol,
                    'Signal': signal,
                    'Net_OI_Change': net_buildup,
                    'Call_Buildup': result['total_call_buildup'],
                    'Put_Buildup': result['total_put_buildup']
                })
    
    if not results:
        print("⚠️  Not enough data yet.\n")
        return None
    
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values('Net_OI_Change', key=abs, ascending=False)
    
    print("Top 10 Stocks with Significant OI Changes:")
    print(results_df.head(10).to_string(index=False))
    
    print("\n💡 Interpretation:")
    print("  • Put Buildup (Positive) = Sellers expect support = BULLISH")
    print("  • Call Buildup (Negative) = Sellers expect resistance = BEARISH")
    print("  • Large changes indicate strong institutional positioning")
    
    results_df.to_csv(os.path.join(ANALYSIS_OUTPUT, f"oi_patterns_{TODAY}.csv"), index=False)
    print(f"\n💾 Saved to: oi_patterns_{TODAY}.csv\n")
    
    return results_df

# ==========================
# 4. SIGNAL WIN RATE CALCULATOR
# ==========================
def calculate_signal_win_rate(symbol, expiry):
    """
    Backtest: When we gave a BULLISH/BEARISH signal, did it work?
    
    This is your MONEY METRIC!
    """
    # Load historical signals (from Cell 2's historical tracking)
    hist_signal_file = os.path.join(OUTPUT_DIR, "historical_tracking", f"{symbol}_history.csv")
    
    if not os.path.exists(hist_signal_file):
        return None
    
    df = pd.read_csv(hist_signal_file)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df[df['Expiry'] == expiry].sort_values('Date')
    
    if len(df) < 3:
        return None
    
    # Calculate actual next-day returns
    df['next_day_cmp'] = df['CMP'].shift(-1)
    df['actual_return'] = (df['next_day_cmp'] / df['CMP'] - 1) * 100
    
    # Check if signal was correct
    df['signal_correct'] = False
    df.loc[(df['Final_Signal'].str.contains('BULLISH')) & (df['actual_return'] > 0), 'signal_correct'] = True
    df.loc[(df['Final_Signal'].str.contains('BEARISH')) & (df['actual_return'] < 0), 'signal_correct'] = True
    
    # Win rate by signal type
    bullish_trades = df[df['Final_Signal'].str.contains('BULLISH')]
    bearish_trades = df[df['Final_Signal'].str.contains('BEARISH')]
    
    bullish_win_rate = (bullish_trades['signal_correct'].sum() / len(bullish_trades) * 100) if len(bullish_trades) > 0 else 0
    bearish_win_rate = (bearish_trades['signal_correct'].sum() / len(bearish_trades) * 100) if len(bearish_trades) > 0 else 0
    
    # Win rate by confidence
    high_conf = df[df['Confidence_pct'] > 60]
    low_conf = df[df['Confidence_pct'] <= 60]
    
    high_conf_win_rate = (high_conf['signal_correct'].sum() / len(high_conf) * 100) if len(high_conf) > 0 else 0
    low_conf_win_rate = (low_conf['signal_correct'].sum() / len(low_conf) * 100) if len(low_conf) > 0 else 0
    
    return {
        'symbol': symbol,
        'expiry': expiry,
        'total_signals': len(df) - 1,  # Exclude last day (no next day yet)
        'bullish_win_rate': bullish_win_rate,
        'bearish_win_rate': bearish_win_rate,
        'high_conf_win_rate': high_conf_win_rate,
        'low_conf_win_rate': low_conf_win_rate,
        'overall_win_rate': (df['signal_correct'].sum() / (len(df) - 1) * 100) if len(df) > 1 else 0
    }

def run_win_rate_study():
    """
    Calculate win rates for all tracked symbols
    """
    print("🎯 Signal Win Rate Analysis")
    print("─" * 70)
    print("Backtesting: How accurate were our signals?\n")
    
    hist_files = []
    hist_dir = os.path.join(OUTPUT_DIR, "historical_tracking")
    
    if os.path.exists(hist_dir):
        hist_files = [f for f in os.listdir(hist_dir) if f.endswith('_history.csv')]
    
    if not hist_files:
        print("⚠️  No historical signals yet.")
        print("   Signals are tracked when you run Cell 3-5.")
        print("   Run analysis for a few days to see win rates!\n")
        return None
    
    results = []
    
    for file in hist_files[:20]:
        symbol = file.replace('_history.csv', '')
        
        # Get expiries for this symbol
        hist_df = pd.read_csv(os.path.join(hist_dir, file))
        expiries = hist_df['Expiry'].unique()
        
        for expiry in expiries:
            result = calculate_signal_win_rate(symbol, expiry)
            if result and result['total_signals'] >= 2:
                results.append({
                    'Symbol': symbol,
                    'Signals': result['total_signals'],
                    'Win_Rate_%': result['overall_win_rate'],
                    'Bullish_WR_%': result['bullish_win_rate'],
                    'Bearish_WR_%': result['bearish_win_rate'],
                    'High_Conf_WR_%': result['high_conf_win_rate']
                })
    
    if not results:
        print("⚠️  Need at least 3 days of signals.\n")
        return None
    
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values('Win_Rate_%', ascending=False)
    
    print("Top 10 Best Performing Signals:")
    print(results_df.head(10).to_string(index=False))
    
    print("\n💡 Key Insights:")
    avg_wr = results_df['Win_Rate_%'].mean()
    print(f"  • Average Win Rate: {avg_wr:.1f}%")
    print(f"  • High Confidence Win Rate: {results_df['High_Conf_WR_%'].mean():.1f}%")
    print(f"  • Total Signals Tracked: {results_df['Signals'].sum()}")
    
    if avg_wr > 55:
        print(f"\n🎉 Your signals are profitable! (>{avg_wr:.1f}% win rate)")
    elif avg_wr > 50:
        print(f"\n👍 Your signals are decent ({avg_wr:.1f}% win rate)")
    else:
        print(f"\n⚠️  Signals need improvement ({avg_wr:.1f}% win rate)")
    
    results_df.to_csv(os.path.join(ANALYSIS_OUTPUT, f"win_rates_{TODAY}.csv"), index=False)
    print(f"\n💾 Saved to: win_rates_{TODAY}.csv\n")
    
    return results_df

# ==========================
# MASTER ANALYSIS RUNNER
# ==========================
def run_full_advanced_analysis():
    """
    Run all advanced analyses
    """
    if not check_data_availability():
        return
    
    print("=" * 70)
    print("🚀 RUNNING FULL ADVANCED ANALYSIS")
    print("=" * 70 + "\n")
    
    # 1. PCR Correlation
    pcr_results = run_pcr_correlation_study()
    
    # 2. Max Pain Accuracy
    max_pain_results = run_max_pain_accuracy_study()
    
    # 3. OI Patterns
    oi_results = run_oi_pattern_study()
    
    # 4. Win Rates
    win_rate_results = run_win_rate_study()
    
    print("\n" + "=" * 70)
    print("✅ ADVANCED ANALYSIS COMPLETE!")
    print("=" * 70)
    print(f"\n📂 All reports saved to: {ANALYSIS_OUTPUT}")
    
    print("\n🎯 Next Steps:")
    print("1. Review the CSV files in advanced_analysis folder")
    print("2. Focus on stocks with:")
    print("   • High PCR correlation (predictive)")
    print("   • Low Max Pain distance (accurate)")
    print("   • High win rates (profitable)")
    print("3. After 10+ days, run Cell 8 for ML predictions")
    
    return {
        'pcr': pcr_results,
        'max_pain': max_pain_results,
        'oi': oi_results,
        'win_rates': win_rate_results
    }

# ==========================
# RUN ANALYSIS
# ==========================
print("💡 This cell analyzes your historical data to find what ACTUALLY works!\n")
print("⏰ Run this weekly to track performance over time.\n")

analysis_results = run_full_advanced_analysis()

🔬 ADVANCED HISTORICAL ANALYSIS ENGINE

📅 Analysis Date: 2025-11-11

💡 This cell analyzes your historical data to find what ACTUALLY works!

⏰ Run this weekly to track performance over time.

📊 Data Status:
   • Files: 612
   • Days of data: 3

💡 You have 3 days - basic analysis available
   Collect 5+ days for full ML features!

🚀 RUNNING FULL ADVANCED ANALYSIS

📈 PCR Correlation Study
──────────────────────────────────────────────────────────────────────
Analyzing: Does PCR predict next-day returns?



C:\Users\sarda\AppData\Local\Temp\ipykernel_24240\783482510.py:127: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  category_returns = df.groupby('pcr_category')['next_day_return_pct'].agg(['mean', 'count', 'std'])
C:\Users\sarda\AppData\Local\Temp\ipykernel_24240\783482510.py:127: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  category_returns = df.groupby('pcr_category')['next_day_return_pct'].agg(['mean', 'count', 'std'])
C:\Users\sarda\AppData\Local\Temp\ipykernel_24240\783482510.py:127: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass

Top 10 Stocks by PCR Predictive Power:
    Symbol      Expiry  Correlation Strength
ADANIENSOL 25-Nov-2025          1.0   Strong
  ADANIENT 25-Nov-2025         -1.0   Strong
    360ONE 30-Dec-2025          1.0   Strong
       ABB 25-Nov-2025          1.0   Strong
 ABCAPITAL 30-Dec-2025         -1.0   Strong
ADANIENSOL 30-Dec-2025         -1.0   Strong
  ADANIENT 30-Dec-2025         -1.0   Strong
ADANIGREEN 25-Nov-2025         -1.0   Strong
ADANIGREEN 30-Dec-2025         -1.0   Strong
ADANIPORTS 25-Nov-2025         -1.0   Strong

💡 Interpretation:
  • Negative correlation: High PCR → Price goes UP (contrarian works!)
  • Positive correlation: High PCR → Price goes DOWN (sentiment follows)
  • |Correlation| > 0.5: Strong predictive power

💾 Saved to: pcr_correlation_2025-11-11.csv

🎯 Max Pain Accuracy Study
──────────────────────────────────────────────────────────────────────
Analyzing: Does price converge to Max Pain?

Top 10 Stocks by Max Pain Accuracy (Lowest Distance):
    Symbol   